In [1]:
import multiprocessing
import os
from re import A
from typing import ItemsView

from py import log
os.environ['PYTHONDONTWRITEBYTECODE'] = '1'
from xtquant import xttrader
from xtquant.xttrader import XtQuantTrader, XtQuantTraderCallback
from xtquant.xttype import StockAccount
from xtquant import xtconstant
import pandas_market_calendars as mcal
import pandas as pd
import akshare as ak

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.dates as mdates
def plot(df, column):
    
    # 设置中文字体
    mpl.rcParams['font.sans-serif'] = ['SimHei']  # 指定中文字体为 SimHei
    mpl.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题

    # 假设这是你的DataFrame，其中包含收益

    # 绘制收益曲线
    # 将日期列转换为日期类型
    # df['date'] = pd.to_datetime(df['date'])

    # # 设置日期列为索引
    # df.set_index('date', inplace=True)
    df.index = pd.to_datetime(df.index)
    # 绘制收益曲线
    plt.figure(figsize=(10, 6))  # 设置图表大小
    plt.plot(df.index, df[column], label='Return')
    plt.title('收益曲线')
    plt.xlabel('日期')
    plt.ylabel('收益率')
    plt.legend()  # 显示图例
    plt.grid(True)  # 显示网格线

    # plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    # plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=10))

    plt.show()

d:\anaconda3\envs\python3.8\lib\site-packages\exchange_calendars\exchange_calendar.py:2345: FutureWarning: 'T' is deprecated and will be removed in a future version. Please use 'min' instead of 'T'.
  align: pd.Timedelta | str = pd.Timedelta(1, "T"),


In [2]:
from xtquant import xtdatacenter as xtdc
xtdc.set_token("26e6009f4de3bfb2ae4b89763f255300e96d6912")

print('xtdc.init')
xtdc.init() # 初始化行情模块，加载合约数据，会需要大约十几秒的时间
print('done')
# print('xtdc.listen')
    # addr_list = [
    # '115.231.218.73:55310', 
    # '115.231.218.79:55310', 
    # '42.228.16.211:55300',
    # '42.228.16.210:55300',
    # '36.99.48.20:55300',
    # '36.99.48.21:55300'
    # ]
    # xtdc.set_allow_optmize_address(addr_list)

    # xtdc.set_kline_mirror_enabled(True) 
    
listen_addr = xtdc.listen(port = 58611)
print(f'done, listen_addr:{listen_addr}')

from xtquant import xtdata
xtdata.connect(port=58611)

print('-----连接上了------')
print(xtdata.data_dir)

xtdc.init
done
done, listen_addr:('0.0.0.0', 58611)
***** xtdata连接成功 *****
服务信息: {'tag': 'xtquant', 'version': '1.0'}
服务地址: 127.0.0.1:58611
数据路径: d:\workspace\TradeX\notebook\new_strategy_eval\data\datadir
设置xtdata.enable_hello = False可隐藏此消息

-----连接上了------
None


In [3]:
import tushare as ts
import pandas as pd
import numpy as np
import talib
from datetime import datetime, timedelta

# 初始化Tushare (需替换为你的token)
ts.set_token('c4dc1a3297388071b02491bc968246f502cccd5e761ee84655a0adc0')
pro = ts.pro_api()

In [4]:
import sqlite3
# months = ['202502']
months = ['202409', '202410', '202411', '202412', '202501', '202502', '202503', '202504', '202505', '202506', '202507']
# months = ['202409', '202410']
combined_df = pd.DataFrame()
for month in months:
    conn = sqlite3.connect('D:\workspace\TradeX\ezMoney\sqlite_db\strategy_data.db')
    db_name = 'strategy_data_aftermarket_%s' % month
    query = "select * from %s where strategy_name in ('xc_dx', 'xc_jl', 'xc_jw', 'xc_qb')" % (db_name)
    df = pd.read_sql_query(query, conn)
    combined_df = pd.concat([combined_df, df], axis=0)
combined_df = combined_df.reset_index(drop=True)
combined_df

,id,date_key,strategy_name,sub_strategy_name,stock_code,stock_name,stock_rank,block_category,block_codes,industry_code,...,circulation_market_value,cgyk,htyk,cgyk_value,htyk_value,is_middle_high_open,is_large_high_open,is_small_low_open,is_middle_low_open,is_large_low_open
0,1604,2024-09-02,xc_dx,,603992.XSHG,松霖科技,1,"000026.BKDL,000023.BKDL,000010.BKDL,000007.BKD...","885869.DDBK,885840.DDBK,885617.DDBK,885478.DDB...",,...,8.936767e+09,,,0.0,0.0,0,0,1,0,0
1,1605,2024-09-02,xc_dx,,002671.XSHE,龙泉股份,2,"000008.BKDL,000035.BKDL,000023.BKDL,000011.BKD...","885571.DDBK,885692.DDBK,885641.DDBK,885991.DDB...",,...,2.516181e+09,,,0.0,0.0,0,0,0,0,0
2,1606,2024-09-02,xc_dx,,000008.XSHE,神州高铁,3,"000012.BKDL,000038.BKDL,000023.BKDL,000035.BKD...","886021.DDBK,885867.DDBK,885517.DDBK,883437.DDB...",,...,7.551130e+09,,,0.0,0.0,0,0,0,0,0
3,1607,2024-09-02,xc_dx,,300625.XSHE,三雄极光,4,"000008.BKDL,000026.BKDL,000023.BKDL,000010.BKD...","885869.DDBK,885984.DDBK,885420.DDBK,886072.DDB...",,...,1.934662e+09,,,0.0,0.0,0,0,1,0,0
4,1608,2024-09-02,xc_dx,,000712.XSHE,锦龙股份,5,"000038.BKDL,000023.BKDL,000015.BKDL,000031.BKDL","885620.DDBK,885694.DDBK,885867.DDBK,885570.DDB...",,...,1.142061e+10,,,0.0,0.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259335,56198,2025-07-28,xc_qb,,688218.XSHG,江苏北人,396,"000023.BKDL,000035.BKDL,000015.BKDL,000027.BKD...","885467.DDBK,885517.DDBK,885929.DDBK,885427.DDB...",,...,2.629400e+09,,,0.0,0.0,0,0,0,0,0
259336,56199,2025-07-28,xc_qb,,688099.XSHG,晶晨股份,397,"000012.BKDL,000038.BKDL,000014.BKDL,000023.BKD...","885785.DDBK,883303.DDBK,885545.DDBK,886088.DDB...",,...,3.137204e+10,,,0.0,0.0,0,0,0,0,0
259337,56200,2025-07-28,xc_qb,,601225.XSHG,陕西煤业,398,"000008.BKDL,000038.BKDL,000023.BKDL,000037.BKD...","886021.DDBK,885867.DDBK,883300.DDBK,885914.DDB...",,...,1.968085e+11,"{""tradeTimestamp"":""09:37:06.550"",""lowClose"":20...",,0.0,0.0,0,0,1,0,0
259338,56201,2025-07-28,xc_qb,,872541.BJSE,铁大科技,399,"000018.BKDL,000027.BKDL,000033.BKDL","885517.DDBK,885929.DDBK,885562.DDBK,885736.DDBK",,...,1.533699e+09,"{""tradeTimestamp"":""09:46:13.000"",""lowClose"":16...",,0.0,0.0,0,0,0,0,0


In [5]:
combined_df = combined_df[
    ((combined_df['stock_code'].str.startswith('00')) | (combined_df['stock_code'].str.startswith('60')))
]
combined_df = combined_df.reset_index(drop=True)

In [6]:
combined_df

,id,date_key,strategy_name,sub_strategy_name,stock_code,stock_name,stock_rank,block_category,block_codes,industry_code,...,circulation_market_value,cgyk,htyk,cgyk_value,htyk_value,is_middle_high_open,is_large_high_open,is_small_low_open,is_middle_low_open,is_large_low_open
0,1604,2024-09-02,xc_dx,,603992.XSHG,松霖科技,1,"000026.BKDL,000023.BKDL,000010.BKDL,000007.BKD...","885869.DDBK,885840.DDBK,885617.DDBK,885478.DDB...",,...,8.936767e+09,,,0.0,0.0,0,0,1,0,0
1,1605,2024-09-02,xc_dx,,002671.XSHE,龙泉股份,2,"000008.BKDL,000035.BKDL,000023.BKDL,000011.BKD...","885571.DDBK,885692.DDBK,885641.DDBK,885991.DDB...",,...,2.516181e+09,,,0.0,0.0,0,0,0,0,0
2,1606,2024-09-02,xc_dx,,000008.XSHE,神州高铁,3,"000012.BKDL,000038.BKDL,000023.BKDL,000035.BKD...","886021.DDBK,885867.DDBK,885517.DDBK,883437.DDB...",,...,7.551130e+09,,,0.0,0.0,0,0,0,0,0
3,1608,2024-09-02,xc_dx,,000712.XSHE,锦龙股份,5,"000038.BKDL,000023.BKDL,000015.BKDL,000031.BKDL","885620.DDBK,885694.DDBK,885867.DDBK,885570.DDB...",,...,1.142061e+10,,,0.0,0.0,0,0,0,0,0
4,1610,2024-09-02,xc_dx,,600901.XSHG,江苏金租,7,"000038.BKDL,000023.BKDL,000037.BKDL,000017.BKDL","885520.DDBK,886021.DDBK,886072.DDBK,885867.DDB...",,...,3.624508e+10,,,0.0,0.0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142625,56194,2025-07-28,xc_qb,,600739.XSHG,辽宁成大,392,"000012.BKDL,000038.BKDL,000023.BKDL,000015.BKD...","886021.DDBK,885867.DDBK,885729.DDBK,885520.DDB...",,...,1.873895e+10,"{""tradeTimestamp"":""10:08:00.420"",""lowClose"":12...",,0.0,0.0,0,0,1,0,0
142626,56195,2025-07-28,xc_qb,,600610.XSHG,中毅达,393,"000024.BKDL,000023.BKDL,000037.BKDL,000004.BKD...","885520.DDBK,886021.DDBK,885863.DDBK,886086.DDB...",,...,9.454414e+09,"{""tradeTimestamp"":""09:32:42.470"",""lowClose"":13...",,0.0,0.0,0,0,0,0,0
142627,56196,2025-07-28,xc_qb,,600491.XSHG,龙元建设,394,"000008.BKDL,000023.BKDL,000035.BKDL,000011.BKD...","885520.DDBK,886087.DDBK,885867.DDBK,885832.DDB...",,...,5.430641e+09,"{""tradeTimestamp"":""09:54:21.500"",""lowClose"":3....",,0.0,0.0,0,0,1,0,0
142628,56197,2025-07-28,xc_qb,,002208.XSHE,合肥城建,395,"000038.BKDL,000023.BKDL,000035.BKDL,000015.BKD...","886021.DDBK,885694.DDBK,885749.DDBK,885787.DDB...",,...,5.758115e+09,"{""tradeTimestamp"":""10:09:33.000"",""lowClose"":7....",,0.0,0.0,0,0,1,0,0


In [7]:
def printNanColumns(df):
    nan_ratios = df.isna().mean() * 100  # 转换为百分比

    # 2. 筛选出有 NaN 值的列
    nan_columns = nan_ratios[nan_ratios > 0]

    # 3. 按比例降序排序并格式化输出
    nan_report = nan_columns.sort_values(ascending=False).round(2).to_frame(name='NaN比例(%)')

    print("包含 NaN 的列及其缺失值比例：")
    print(nan_report)

In [8]:
def process_group(group):
    # 初始化各排名字段为默认值 500
    rank_dict = {
        'stock_rank_dx': 500,
        'stock_rank_jl': 500,
        'stock_rank_jw': 500,
        'stock_rank_qb': 500
    }
    
    # 为每个 strategy_name 查找对应的 stock_rank
    for strategy, rank_col in zip(['xc_dx', 'xc_jl', 'xc_jw', 'xc_qb'], 
                                  ['stock_rank_dx', 'stock_rank_jl', 'stock_rank_jw', 'stock_rank_qb']):
        rank = group[group['strategy_name'] == strategy]['stock_rank']
        if not rank.empty:
            rank_dict[rank_col] = rank.iloc[0]
    
    # 随机选择一行保留
    result = group.sample(1)
    # 添加排名字段
    for col, value in rank_dict.items():
        result[col] = value
    return result
combined_df = combined_df.groupby(['date_key', 'stock_code']).apply(process_group).reset_index(drop=True)
combined_df

C:\Users\Administrator\AppData\Local\Temp\ipykernel_84720\596224123.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  combined_df = combined_df.groupby(['date_key', 'stock_code']).apply(process_group).reset_index(drop=True)


,id,date_key,strategy_name,sub_strategy_name,stock_code,stock_name,stock_rank,block_category,block_codes,industry_code,...,htyk_value,is_middle_high_open,is_large_high_open,is_small_low_open,is_middle_low_open,is_large_low_open,stock_rank_dx,stock_rank_jl,stock_rank_jw,stock_rank_qb
0,1630,2024-09-02,xc_dx,,000001.XSHE,平安银行,27,"000003.BKDL,000038.BKDL,000023.BKDL,000037.BKDL","885694.DDBK,885867.DDBK,885745.DDBK,883300.DDB...",,...,0.0,0,0,0,0,0,27,500,500,103
1,1606,2024-09-02,xc_dx,,000008.XSHE,神州高铁,3,"000012.BKDL,000038.BKDL,000023.BKDL,000035.BKD...","886021.DDBK,885867.DDBK,885517.DDBK,883437.DDB...",,...,0.0,0,0,0,0,0,3,500,500,307
2,1971,2024-09-02,xc_dx,,000026.XSHE,飞亚达,368,"000023.BKDL,000014.BKDL,000010.BKDL,000017.BKDL","885420.DDBK,886021.DDBK,886072.DDBK,885595.DDB...",,...,0.0,0,0,1,0,0,368,500,19,52
3,16594,2024-09-02,xc_qb,,000058.XSHE,深 赛 格,342,"000012.BKDL,000038.BKDL,000023.BKDL,000035.BKD...","886021.DDBK,885574.DDBK,885729.DDBK,885697.DDB...",,...,0.0,0,0,1,0,0,39,500,500,342
4,9542,2024-09-02,xc_jw,,000065.XSHE,北方国际,358,"000008.BKDL,000038.BKDL,000026.BKDL,000035.BKD...","886045.DDBK,885996.DDBK,885840.DDBK,886021.DDB...",,...,0.0,0,0,1,0,0,500,500,358,500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87436,55267,2025-07-28,xc_jw,,605298.XSHG,必得科技,231,"000018.BKDL,000033.BKDL","885929.DDBK,885562.DDBK,885427.DDBK",,...,0.0,0,0,1,0,0,500,500,231,500
87437,54366,2025-07-28,xc_dx,,605366.XSHG,宏柏新材,115,"000036.BKDL,000023.BKDL,000035.BKDL,000037.BKD...","885520.DDBK,885912.DDBK,886089.DDBK,885958.DDB...",,...,0.0,0,0,0,0,0,115,500,500,500
87438,56024,2025-07-28,xc_qb,,605398.XSHG,新炬网络,222,"000012.BKDL,000003.BKDL,000026.BKDL,000006.BKD...","885980.DDBK,885832.DDBK,885611.DDBK,885887.DDB...",,...,0.0,0,0,0,0,0,500,500,500,222
87439,54486,2025-07-28,xc_dx,,605488.XSHG,福莱新材,235,"000036.BKDL,000023.BKDL,000006.BKDL,000029.BKD...","885869.DDBK,885806.DDBK,885800.DDBK,885929.DDB...",,...,0.0,0,0,0,0,0,235,500,500,500


In [9]:
combined_df  = combined_df .drop(columns=['strategy_name', 'sub_strategy_name', 'stock_rank'])

In [10]:
combined_df  = combined_df .drop(columns=['created_at', 'block_category_info'])

In [11]:
combined_df = combined_df.drop(columns=['htyk', 'cgyk', 'mod_code', 'mod_name'])

In [12]:
# 定义列名与对应排名字段的映射关系
column_mapping = {
    'cjs': 'stock_rank_dx',
    'xcjw': 'stock_rank_jw',
    'jsjl': 'stock_rank_jl',
    'jssb': 'stock_rank_qb'
}

# 遍历每个列名及其对应的排名字段
for col, rank_col in column_mapping.items():
    # 根据对应列是否为 0 更新排名字段
    combined_df[rank_col] = combined_df.apply(
        lambda row: row[rank_col] if row[col] != 0 else 500, axis=1
    )

combined_df

,id,date_key,stock_code,stock_name,block_category,block_codes,industry_code,max_block_category_rank,max_block_code_rank,max_industry_code_rank,...,htyk_value,is_middle_high_open,is_large_high_open,is_small_low_open,is_middle_low_open,is_large_low_open,stock_rank_dx,stock_rank_jl,stock_rank_jw,stock_rank_qb
0,1630,2024-09-02,000001.XSHE,平安银行,"000003.BKDL,000038.BKDL,000023.BKDL,000037.BKDL","885694.DDBK,885867.DDBK,885745.DDBK,883300.DDB...",,7,2,-1,...,0.0,0,0,0,0,0,500,500,500,500
1,1606,2024-09-02,000008.XSHE,神州高铁,"000012.BKDL,000038.BKDL,000023.BKDL,000035.BKD...","886021.DDBK,885867.DDBK,885517.DDBK,883437.DDB...",,2,1,-1,...,0.0,0,0,0,0,0,500,500,500,500
2,1971,2024-09-02,000026.XSHE,飞亚达,"000023.BKDL,000014.BKDL,000010.BKDL,000017.BKDL","885420.DDBK,886021.DDBK,886072.DDBK,885595.DDB...",,10,21,-1,...,0.0,0,0,1,0,0,500,500,500,500
3,16594,2024-09-02,000058.XSHE,深 赛 格,"000012.BKDL,000038.BKDL,000023.BKDL,000035.BKD...","886021.DDBK,885574.DDBK,885729.DDBK,885697.DDB...",,2,1,-1,...,0.0,0,0,1,0,0,500,500,500,500
4,9542,2024-09-02,000065.XSHE,北方国际,"000008.BKDL,000038.BKDL,000026.BKDL,000035.BKD...","886045.DDBK,885996.DDBK,885840.DDBK,886021.DDB...",,6,2,-1,...,0.0,0,0,1,0,0,500,500,500,500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87436,55267,2025-07-28,605298.XSHG,必得科技,"000018.BKDL,000033.BKDL","885929.DDBK,885562.DDBK,885427.DDBK",,3,10,-1,...,0.0,0,0,1,0,0,500,500,231,500
87437,54366,2025-07-28,605366.XSHG,宏柏新材,"000036.BKDL,000023.BKDL,000035.BKDL,000037.BKD...","885520.DDBK,885912.DDBK,886089.DDBK,885958.DDB...",,1,34,-1,...,0.0,0,0,0,0,0,500,500,500,500
87438,56024,2025-07-28,605398.XSHG,新炬网络,"000012.BKDL,000003.BKDL,000026.BKDL,000006.BKD...","885980.DDBK,885832.DDBK,885611.DDBK,885887.DDB...",,5,2,-1,...,0.0,0,0,0,0,0,500,500,500,222
87439,54486,2025-07-28,605488.XSHG,福莱新材,"000036.BKDL,000023.BKDL,000006.BKDL,000029.BKD...","885869.DDBK,885806.DDBK,885800.DDBK,885929.DDB...",,1,2,-1,...,0.0,0,0,0,0,0,500,500,500,500


In [13]:
combined_df.tail(40)

,id,date_key,stock_code,stock_name,block_category,block_codes,industry_code,max_block_category_rank,max_block_code_rank,max_industry_code_rank,...,htyk_value,is_middle_high_open,is_large_high_open,is_small_low_open,is_middle_low_open,is_large_low_open,stock_rank_dx,stock_rank_jl,stock_rank_jw,stock_rank_qb
87401,55884,2025-07-28,603613.XSHG,国联股份,"000012.BKDL,000003.BKDL,000038.BKDL,000026.BKD...","886100.DDBK,883303.DDBK,886041.DDBK,885629.DDB...",,5,2,-1,...,0.0,1,0,0,0,0,500,500,207,82
87402,55063,2025-07-28,603616.XSHG,韩建河山,"000008.BKDL,000024.BKDL,000018.BKDL,000011.BKD...","885692.DDBK,885962.DDBK,885690.DDBK,885563.DDB...",,14,84,-1,...,0.0,0,0,0,0,1,10,500,27,500
87403,54628,2025-07-28,603619.XSHG,中曼石油,"000026.BKDL,000023.BKDL,000018.BKDL,000037.BKD...","885869.DDBK,885520.DDBK,885840.DDBK,885867.DDB...",,1,5,-1,...,0.0,0,0,0,0,0,500,500,500,500
87404,54545,2025-07-28,603626.XSHG,科森科技,"000012.BKDL,000014.BKDL,000023.BKDL,000035.BKD...","885899.DDBK,885771.DDBK,885867.DDBK,885467.DDB...",,1,2,-1,...,0.0,0,0,0,0,0,500,500,500,500
87405,55853,2025-07-28,603639.XSHG,海利尔,"000023.BKDL,000018.BKDL,000037.BKDL","885520.DDBK,886072.DDBK,885867.DDBK,885494.DDBK",,1,6,-1,...,0.0,1,0,0,0,0,500,500,500,51
87406,55928,2025-07-28,603663.XSHG,三祥新材,"000036.BKDL,000035.BKDL,000027.BKDL,000001.BKD...","885571.DDBK,885929.DDBK,886000.DDBK,885343.DDB...",,2,10,-1,...,0.0,0,0,0,0,0,500,500,500,126
87407,54477,2025-07-28,603676.XSHG,卫信康,"000037.BKDL,000004.BKDL,000009.BKDL","885520.DDBK,886086.DDBK,886081.DDBK,885403.DDBK",,10,11,-1,...,0.0,0,0,0,1,0,500,500,181,500
87408,56058,2025-07-28,603681.XSHG,永冠新材,"000026.BKDL,000023.BKDL,000037.BKDL,000004.BKD...","885520.DDBK,885840.DDBK,885901.DDBK,883303.DDB...",,1,5,-1,...,0.0,0,0,0,0,0,500,500,500,256
87409,54488,2025-07-28,603716.XSHG,塞力医疗,"000003.BKDL,000006.BKDL,000019.BKDL,000016.BKD...","885818.DDBK,886100.DDBK,885993.DDBK,885459.DDB...",,5,3,-1,...,0.0,0,0,0,1,0,500,500,500,500
87410,55988,2025-07-28,603738.XSHG,泰晶科技,"000035.BKDL,000006.BKDL,000027.BKDL,000031.BKD...","885843.DDBK,885517.DDBK,883303.DDBK,885929.DDB...",,2,2,-1,...,0.0,0,0,0,0,0,500,500,500,186


In [14]:
import sys
sys.path.append(r"D:\workspace\TradeX\ezMoney")
from http_request import build_http_request
from data_class import category_rank_class
def get_ranked_category_infos(date_key, except_is_ppp = True, except_is_track = True, gap = 10):
    # build_http_request.check_user_alive()
    categoryRankList = category_rank_class.build_category_rank_sort_list(date_key)
    block_rank_list = []
    for item in categoryRankList:
        if item == None:
            continue
        if item.stockType == 'industry':
            isPpp = item.isPpp
            num = item.num
            numChange = item.numChange
            isTrack = item.isTrack
            stockCode = item.categoryCode
            r  = {
                'isPpp': isPpp,
                'num': num,
                'numChange': numChange,
                'isTrack': isTrack,
                'blockCode': stockCode
            }
            block_rank_list.append(r)
        if item.blockRankList == None:
            continue
        block_rank_list.extend(item.blockRankList)

    if except_is_ppp:
        block_rank_list = [item for item in block_rank_list if not item['isPpp']]
    if except_is_track:
        block_rank_list = [item for item in block_rank_list if not item['isTrack']]

    sorted_block_rank_list = sorted(block_rank_list, key=lambda x: (-x['num'], -x['numChange']))
    rank_dict = {}
    prev_num = None
    current_rank = 1

    for idx, item in enumerate(sorted_block_rank_list):
        code = item['blockCode']
        num = item['num']

        if idx == 0:
            # 第一个元素直接赋初始排名
            rank_dict[code] = current_rank
            prev_num = num
            continue

        delta = 0
        diff = prev_num - num

        # 判断差值规则
        if abs(diff) <= 0.0001:
            delta = 0
        else:
            if gap == 0:
                delta = 1
            else:
                if diff > gap:
                    delta = int(diff // gap) + 1
                else:
                    delta = 1
        current_rank += delta
        rank_dict[code] = current_rank
        prev_num = num
    return rank_dict

# import sys
# sys.path.append(r"D:\workspace\TradeX\ezMoney")
# from http_request import build_http_request
# from data_class import category_rank_class
def get_ranked_new_category_infos(date_key, except_is_ppp = True, except_is_track = True, gap = 10):
    # build_http_request.check_user_alive()
    categoryRankList = category_rank_class.build_category_rank_sort_list(date_key)
    if except_is_ppp:
        categoryRankList = [item for item in categoryRankList if not item.isPpp]
    if except_is_track:
        categoryRankList = [item for item in categoryRankList if not item.isTrack]
    

    sorted_block_rank_list = sorted(categoryRankList, key=lambda x: (-x.num, -x.numChange))
    rank_dict = {}
    prev_num = None
    current_rank = 1

    for idx, item in enumerate(sorted_block_rank_list):
        code = item.categoryCode
        num = item.num

        if idx == 0:
            # 第一个元素直接赋初始排名
            rank_dict[code] = current_rank
            prev_num = num
            continue

        delta = 0
        diff = prev_num - num

        # 判断差值规则
        if abs(diff) <= 0.0001:
            delta = 0
        else:
            if gap == 0:
                delta = 1
            else:
                if diff > gap:
                    delta = int(diff // gap) + 1
                else:
                    delta = 1
        current_rank += delta
        rank_dict[code] = current_rank
        prev_num = num
    return rank_dict

In [15]:
import sys
sys.path.append(r"D:\workspace\TradeX\ezMoney")
from date_utils import date

def get_previous_trade_date_n(current_date, n = 3):
    res_dates = []
    for i in range(n):
        current_date = date.get_previous_trade_date(current_date)
        res_dates.append(current_date)
    return res_dates

In [16]:
from datetime import datetime, timedelta
from date_utils import date
import numpy as np

# 缓存字典
block_rank_cache = {}
category_rank_cache = {}

def get_previous_dates(date_key, days=3):
    return get_previous_trade_date_n(date_key, days)

# 四种参数组合
param_combinations = [
    {'except_is_ppp': True, 'except_is_track': True},
    {'except_is_ppp': True, 'except_is_track': False},
    {'except_is_ppp': False, 'except_is_track': True},
    {'except_is_ppp': False, 'except_is_track': False}
]

# 添加新列
for params in param_combinations:
    ppp_key = 'T' if params['except_is_ppp'] else 'F'
    track_key = 'T' if params['except_is_track'] else 'F'
    prefix = f"ppp_{ppp_key}_track_{track_key}"
    
    # 当天排名字段
    combined_df[f'top1_rank_{prefix}'] = 100
    combined_df[f'top2_rank_{prefix}'] = 100
    combined_df[f'top3_rank_{prefix}'] = 100
    combined_df[f'top3_avgrank_{prefix}'] = 100
    combined_df[f'top1_category_rank_{prefix}'] = 100
    combined_df[f'top3_avg_category_rank_{prefix}'] = 100
    
    # 前三天平均排名字段
    combined_df[f'top1_predays_avgrank_{prefix}'] = 100
    combined_df[f'top2_predays_avgrank_{prefix}'] = 100
    combined_df[f'top3_predays_avgrank_{prefix}'] = 100
    combined_df[f'top1_category_predays_avgrank_{prefix}'] = 100
    combined_df[f'top3_avg_category_predays_avgrank_{prefix}'] = 100
# 主处理循环
for idx, row in combined_df.iterrows():
    date_key = row['date_key']
    block_codes = row['block_codes'].split(',') if row['block_codes'] else []
    industry_codes = row['industry_code'].split(',') if row['industry_code'] else []
    block_categories = row['block_category'].split(',') if row['block_category'] else []
    
    # 获取前三天的日期
    prev_dates = get_previous_dates(date_key)
    
    for params in param_combinations:
        ppp = params['except_is_ppp']
        track = params['except_is_track']
        ppp_key = 'T' if ppp else 'F'
        track_key = 'T' if track else 'F'
        prefix = f"ppp_{ppp_key}_track_{track_key}"
        
        # === 当天排名计算 ===
        # 获取当天排名字典（使用缓存）
        cache_key = (date_key, ppp, track)
        if cache_key not in block_rank_cache:
            block_rank_cache[cache_key] = get_ranked_category_infos(
                date_key, gap=0, except_is_ppp=ppp, except_is_track=track)
        if cache_key not in category_rank_cache:
            category_rank_cache[cache_key] = get_ranked_new_category_infos(
                date_key, gap=0, except_is_ppp=ppp, except_is_track=track)
        
        ranked_block = block_rank_cache[cache_key]
        ranked_category = category_rank_cache[cache_key]
        
        # 计算block+industry排名
        all_ranks = []
        for code in block_codes + industry_codes:
            if code in ranked_block:
                all_ranks.append(ranked_block[code])
        all_ranks = sorted(all_ranks)[:3]  # 取TOP3
        if all_ranks:
            combined_df.loc[idx, f'top1_rank_{prefix}'] = all_ranks[0]
            combined_df.loc[idx, f'top2_rank_{prefix}'] = all_ranks[1] if len(all_ranks) > 1 else 100
            combined_df.loc[idx, f'top3_rank_{prefix}'] = all_ranks[2] if len(all_ranks) > 2 else 100
            combined_df.loc[idx, f'top3_avgrank_{prefix}'] = sum(all_ranks[:3]) / min(3, len(all_ranks))
        
        # 计算block category排名
        cat_ranks = []
        for cat in block_categories:
            if cat in ranked_category:
                cat_ranks.append(ranked_category[cat])
        cat_ranks = sorted(cat_ranks)[:3]  # 取TOP3
        if cat_ranks:
            combined_df.loc[idx, f'top1_category_rank_{prefix}'] = cat_ranks[0]
            combined_df.loc[idx, f'top3_avg_category_rank_{prefix}'] = sum(cat_ranks[:3]) / min(3, len(cat_ranks))
        
        # === 前三天平均排名计算 ===
        top1_list, top2_list, top3_list = [], [], []
        cat_top1_list, cat_avg_list = [], []
        
        for p_date in prev_dates:
            # 获取历史排名字典（使用缓存）
            p_cache_key = (p_date, ppp, track)
            try:
                if p_cache_key not in block_rank_cache:
                    block_rank_cache[p_cache_key] = get_ranked_category_infos(
                        p_date, gap=0, except_is_ppp=ppp, except_is_track=track)
                if p_cache_key not in category_rank_cache:
                    category_rank_cache[p_cache_key] = get_ranked_new_category_infos(
                        p_date, gap=0, except_is_ppp=ppp, except_is_track=track)
                
                p_ranked_block = block_rank_cache[p_cache_key]
                p_ranked_category = category_rank_cache[p_cache_key]
                
                # 计算历史日期的block+industry排名
                p_ranks = []
                for code in block_codes + industry_codes:
                    if code in p_ranked_block:
                        p_ranks.append(p_ranked_block[code])
                p_ranks = sorted(p_ranks)[:3]
                if p_ranks:
                    top1_list.append(p_ranks[0])
                    top2_list.append(p_ranks[1] if len(p_ranks) > 1 else 100)
                    top3_list.append(p_ranks[2] if len(p_ranks) > 2 else 100)
                
                # 计算历史日期的block category排名
                p_cat_ranks = []
                for cat in block_categories:
                    if cat in p_ranked_category:
                        p_cat_ranks.append(p_ranked_category[cat])
                p_cat_ranks = sorted(p_cat_ranks)[:3]
                if p_cat_ranks:
                    cat_top1_list.append(p_cat_ranks[0])
                    cat_avg_list.append(sum(p_cat_ranks[:3]) / min(3, len(p_cat_ranks)))
                    
            except Exception as e:
                print(f"Error processing {p_date}: {str(e)}")
                continue
        
        # 计算前三天的平均值
        if top1_list:
            combined_df.loc[idx, f'top1_predays_avgrank_{prefix}'] = sum(top1_list) / len(top1_list)
        if top2_list:
            combined_df.loc[idx, f'top2_predays_avgrank_{prefix}'] = sum(top2_list) / len(top2_list)
        if top3_list:
            combined_df.loc[idx, f'top3_predays_avgrank_{prefix}'] = sum(top3_list) / len(top3_list)
        if cat_top1_list:
            combined_df.loc[idx, f'top1_category_predays_avgrank_{prefix}'] = sum(cat_top1_list) / len(cat_top1_list)
        if cat_avg_list:
            combined_df.loc[idx, f'top3_avg_category_predays_avgrank_{prefix}'] = sum(cat_avg_list) / len(cat_avg_list)

# 清理缓存
block_rank_cache.clear()
category_rank_cache.clear()

C:\Users\Administrator\AppData\Local\Temp\ipykernel_84720\576307305.py:143: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '5.666666666666667' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  combined_df.loc[idx, f'top1_category_predays_avgrank_{prefix}'] = sum(cat_top1_list) / len(cat_top1_list)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_84720\576307305.py:90: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '10.333333333333334' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  combined_df.loc[idx, f'top3_avg_category_rank_{prefix}'] = sum(cat_ranks[:3]) / min(3, len(cat_ranks))
C:\Users\Administrator\AppData\Local\Temp\ipykernel_84720\576307305.py:145: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an erro

In [17]:
combined_df

,id,date_key,stock_code,stock_name,block_category,block_codes,industry_code,max_block_category_rank,max_block_code_rank,max_industry_code_rank,...,top2_rank_ppp_F_track_F,top3_rank_ppp_F_track_F,top3_avgrank_ppp_F_track_F,top1_category_rank_ppp_F_track_F,top3_avg_category_rank_ppp_F_track_F,top1_predays_avgrank_ppp_F_track_F,top2_predays_avgrank_ppp_F_track_F,top3_predays_avgrank_ppp_F_track_F,top1_category_predays_avgrank_ppp_F_track_F,top3_avg_category_predays_avgrank_ppp_F_track_F
0,1630,2024-09-02,000001.XSHE,平安银行,"000003.BKDL,000038.BKDL,000023.BKDL,000037.BKDL","885694.DDBK,885867.DDBK,885745.DDBK,883300.DDB...",,7,2,-1,...,100,100,2.000000,6,10.333333,3.000000,44.333333,100.000000,6.000000,9.000000
1,1606,2024-09-02,000008.XSHE,神州高铁,"000012.BKDL,000038.BKDL,000023.BKDL,000035.BKD...","886021.DDBK,885867.DDBK,885517.DDBK,883437.DDB...",,2,1,-1,...,2,5,2.666667,2,3.000000,1.000000,2.333333,5.666667,2.000000,2.777778
2,1971,2024-09-02,000026.XSHE,飞亚达,"000023.BKDL,000014.BKDL,000010.BKDL,000017.BKDL","885420.DDBK,886021.DDBK,886072.DDBK,885595.DDB...",,10,21,-1,...,11,13,10.666667,7,9.333333,7.666667,9.000000,13.000000,2.333333,5.333333
3,16594,2024-09-02,000058.XSHE,深 赛 格,"000012.BKDL,000038.BKDL,000023.BKDL,000035.BKD...","886021.DDBK,885574.DDBK,885729.DDBK,885697.DDB...",,2,1,-1,...,2,8,3.666667,2,3.666667,1.333333,3.333333,8.000000,2.000000,2.888889
4,9542,2024-09-02,000065.XSHE,北方国际,"000008.BKDL,000038.BKDL,000026.BKDL,000035.BKD...","886045.DDBK,885996.DDBK,885840.DDBK,886021.DDB...",,6,2,-1,...,7,8,5.666667,5,6.666667,3.000000,6.000000,9.666667,3.666667,6.111111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87436,55267,2025-07-28,605298.XSHG,必得科技,"000018.BKDL,000033.BKDL","885929.DDBK,885562.DDBK,885427.DDBK",,3,10,-1,...,16,100,10.500000,3,11.500000,6.333333,16.333333,30.333333,4.333333,7.166667
87437,54366,2025-07-28,605366.XSHG,宏柏新材,"000036.BKDL,000023.BKDL,000035.BKDL,000037.BKD...","885520.DDBK,885912.DDBK,886089.DDBK,885958.DDB...",,1,34,-1,...,19,19,17.000000,1,7.333333,9.666667,18.333333,29.666667,1.000000,7.777778
87438,56024,2025-07-28,605398.XSHG,新炬网络,"000012.BKDL,000003.BKDL,000026.BKDL,000006.BKD...","885980.DDBK,885832.DDBK,885611.DDBK,885887.DDB...",,5,2,-1,...,2,15,6.333333,5,5.666667,15.000000,17.000000,22.000000,2.666667,8.555556
87439,54486,2025-07-28,605488.XSHG,福莱新材,"000036.BKDL,000023.BKDL,000006.BKDL,000029.BKD...","885869.DDBK,885806.DDBK,885800.DDBK,885929.DDB...",,1,2,-1,...,5,9,5.333333,1,3.000000,6.333333,14.000000,17.333333,1.333333,7.555556


In [18]:
printNanColumns(combined_df)

包含 NaN 的列及其缺失值比例：
Empty DataFrame
Columns: [NaN比例(%)]
Index: []


In [19]:
all_stocks = {}
all_stocks_info = xtdata.get_stock_list_in_sector('沪深A股')
for stock in all_stocks_info:
    if stock.startswith('60') or stock.startswith('00'):
        cde = stock.split('.')[0]
        all_stocks[cde] = stock

In [20]:

def map_stock_code(row):
    stock_code = row['stock_code']
    prefix = stock_code.split('.')[0]
    try:
        # 尝试从 all_stocks 中获取对应值
        return all_stocks[prefix]
    except KeyError:
        # 若键不存在，打印 stock_code 并返回 None
        print(f"未找到对应的 code: {stock_code}")
        return None

# 应用映射函数到每一行，返回映射后的值或 None
combined_df['stock_code'] = combined_df.apply(map_stock_code, axis=1)

# 删除 stockcode 列为 None 的行
combined_df = combined_df.dropna(subset=['stock_code'])

# 重置索引
combined_df = combined_df.reset_index(drop=True)

# 打印处理后的 DataFrame
print(combined_df)

未找到对应的 code: 000584.XSHE
未找到对应的 code: 600462.XSHG
未找到对应的 code: 000584.XSHE
未找到对应的 code: 600462.XSHG
未找到对应的 code: 000584.XSHE
未找到对应的 code: 600462.XSHG
未找到对应的 code: 000584.XSHE
未找到对应的 code: 600462.XSHG
未找到对应的 code: 000584.XSHE
未找到对应的 code: 600462.XSHG
未找到对应的 code: 000584.XSHE
未找到对应的 code: 600462.XSHG
未找到对应的 code: 000584.XSHE
未找到对应的 code: 600462.XSHG
未找到对应的 code: 000584.XSHE
未找到对应的 code: 600462.XSHG
未找到对应的 code: 000584.XSHE
未找到对应的 code: 600462.XSHG
未找到对应的 code: 000584.XSHE
未找到对应的 code: 600462.XSHG
未找到对应的 code: 000584.XSHE
未找到对应的 code: 600462.XSHG
未找到对应的 code: 000584.XSHE
未找到对应的 code: 600462.XSHG
未找到对应的 code: 000584.XSHE
未找到对应的 code: 600462.XSHG
未找到对应的 code: 000584.XSHE
未找到对应的 code: 600462.XSHG
未找到对应的 code: 000584.XSHE
未找到对应的 code: 600462.XSHG
未找到对应的 code: 000584.XSHE
未找到对应的 code: 600462.XSHG
未找到对应的 code: 000584.XSHE
未找到对应的 code: 600462.XSHG
未找到对应的 code: 000584.XSHE
未找到对应的 code: 600462.XSHG
未找到对应的 code: 000584.XSHE
未找到对应的 code: 600462.XSHG
未找到对应的 code: 000584.XSHE
未找到对应的 code: 600462.XSHG


In [21]:
combined_df

,id,date_key,stock_code,stock_name,block_category,block_codes,industry_code,max_block_category_rank,max_block_code_rank,max_industry_code_rank,...,top2_rank_ppp_F_track_F,top3_rank_ppp_F_track_F,top3_avgrank_ppp_F_track_F,top1_category_rank_ppp_F_track_F,top3_avg_category_rank_ppp_F_track_F,top1_predays_avgrank_ppp_F_track_F,top2_predays_avgrank_ppp_F_track_F,top3_predays_avgrank_ppp_F_track_F,top1_category_predays_avgrank_ppp_F_track_F,top3_avg_category_predays_avgrank_ppp_F_track_F
0,1630,2024-09-02,000001.SZ,平安银行,"000003.BKDL,000038.BKDL,000023.BKDL,000037.BKDL","885694.DDBK,885867.DDBK,885745.DDBK,883300.DDB...",,7,2,-1,...,100,100,2.000000,6,10.333333,3.000000,44.333333,100.000000,6.000000,9.000000
1,1606,2024-09-02,000008.SZ,神州高铁,"000012.BKDL,000038.BKDL,000023.BKDL,000035.BKD...","886021.DDBK,885867.DDBK,885517.DDBK,883437.DDB...",,2,1,-1,...,2,5,2.666667,2,3.000000,1.000000,2.333333,5.666667,2.000000,2.777778
2,1971,2024-09-02,000026.SZ,飞亚达,"000023.BKDL,000014.BKDL,000010.BKDL,000017.BKDL","885420.DDBK,886021.DDBK,886072.DDBK,885595.DDB...",,10,21,-1,...,11,13,10.666667,7,9.333333,7.666667,9.000000,13.000000,2.333333,5.333333
3,16594,2024-09-02,000058.SZ,深 赛 格,"000012.BKDL,000038.BKDL,000023.BKDL,000035.BKD...","886021.DDBK,885574.DDBK,885729.DDBK,885697.DDB...",,2,1,-1,...,2,8,3.666667,2,3.666667,1.333333,3.333333,8.000000,2.000000,2.888889
4,9542,2024-09-02,000065.SZ,北方国际,"000008.BKDL,000038.BKDL,000026.BKDL,000035.BKD...","886045.DDBK,885996.DDBK,885840.DDBK,886021.DDB...",,6,2,-1,...,7,8,5.666667,5,6.666667,3.000000,6.000000,9.666667,3.666667,6.111111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87390,55267,2025-07-28,605298.SH,必得科技,"000018.BKDL,000033.BKDL","885929.DDBK,885562.DDBK,885427.DDBK",,3,10,-1,...,16,100,10.500000,3,11.500000,6.333333,16.333333,30.333333,4.333333,7.166667
87391,54366,2025-07-28,605366.SH,宏柏新材,"000036.BKDL,000023.BKDL,000035.BKDL,000037.BKD...","885520.DDBK,885912.DDBK,886089.DDBK,885958.DDB...",,1,34,-1,...,19,19,17.000000,1,7.333333,9.666667,18.333333,29.666667,1.000000,7.777778
87392,56024,2025-07-28,605398.SH,新炬网络,"000012.BKDL,000003.BKDL,000026.BKDL,000006.BKD...","885980.DDBK,885832.DDBK,885611.DDBK,885887.DDB...",,5,2,-1,...,2,15,6.333333,5,5.666667,15.000000,17.000000,22.000000,2.666667,8.555556
87393,54486,2025-07-28,605488.SH,福莱新材,"000036.BKDL,000023.BKDL,000006.BKDL,000029.BKD...","885869.DDBK,885806.DDBK,885800.DDBK,885929.DDB...",,1,2,-1,...,5,9,5.333333,1,3.000000,6.333333,14.000000,17.333333,1.333333,7.555556


In [ ]:
import tushare as ts
import pandas as pd
import numpy as np
import talib
from datetime import datetime, timedelta

# 初始化Tushare (需替换为你的token)
ts.set_token('c4dc1a3297388071b02491bc968246f502cccd5e761ee84655a0adc0')
pro = ts.pro_api()

# ===================== 数据准备 =====================
def prepare_data(ts_code='600519.SH', start_date='20200101', end_date='20231231'):
    """
    获取并预处理股票数据
    """
    # 获取日线行情
    # 获取日线行情
    df_daily = pro.daily(ts_code=ts_code, start_date=start_date, end_date=end_date)
    df_daily = df_daily.sort_values('trade_date').reset_index(drop=True)
    
    # 转换为数值类型
    num_cols = ['open', 'high', 'low', 'close', 'vol', 'amount', 'pre_close']
    df_daily[num_cols] = df_daily[num_cols].apply(pd.to_numeric)
    
    # 获取daily_basic数据
    df_basic = pro.daily_basic(
        ts_code=ts_code, 
        start_date=start_date, 
        end_date=end_date,
        fields='trade_date,turnover_rate,turnover_rate_f,volume_ratio,pe,pe_ttm,pb,ps,ps_ttm,dv_ratio,dv_ttm,total_share,float_share,free_share,total_mv,circ_mv'
    )
    
    # 转换为数值类型
    basic_num_cols = ['turnover_rate', 'turnover_rate_f', 'volume_ratio', 'pe', 'pe_ttm', 
                     'pb', 'ps', 'ps_ttm', 'dv_ratio', 'dv_ttm', 'total_share', 
                     'float_share', 'free_share', 'total_mv', 'circ_mv']
    df_basic[basic_num_cols] = df_basic[basic_num_cols].apply(pd.to_numeric)
    
    # 合并数据
    df = pd.merge(df_daily, df_basic, on=['trade_date'], how='left')
    
    # 添加技术指标
    df['ma5'] = df['close'].rolling(5).mean()
    df['ma10'] = df['close'].rolling(10).mean()
    df['ma20'] = df['close'].rolling(20).mean()
    
    # 添加换手率相关衍生指标
    df['turnover_rate_5d'] = df['turnover_rate'].rolling(5).mean()
    df['turnover_rate_10d'] = df['turnover_rate'].rolling(10).mean()

    # 填充所有NaN为0
    df = df.fillna(0)
    # 替换无穷值
    df = df.replace([np.inf, -np.inf], 0)
    
    return df

# ===================== 因子计算 =====================
class FactorGenerator:
    """量价因子生成器"""
    
    def __init__(self, data):
        self.data = data.copy()
        self.calc_all_factors()
    
    def calc_all_factors(self):
        # # 1. 计算基础量价因子
        # self.calc_basic_factors()
        
        # # 2. 计算技术形态因子
        # self.calc_technical_factors()
        
        # # 3. 计算事件驱动因子
        # self.calc_event_factors()
        
        # # 4. 计算趋势相关因子
        # self.calc_trend_factors()
        
        # self.calc_additional_factors()

        # # 5. 计算daily_basic相关因子
        # self.calc_daily_basic_factors()
        self.calc_basic_factors()
        self.calc_technical_factors()
        self.calc_event_factors()
        self.calc_trend_factors()
        self.calc_additional_factors()
        self.calc_daily_basic_factors()
        # 最终清理
        self.data = self.data.fillna(0)
        self.data = self.data.replace([np.inf, -np.inf], 0)
        return self.data
    
    def calc_basic_factors(self):
        """基础量价因子计算"""
        df = self.data
        # 添加ma3计算
        df['ma3'] = df['close'].rolling(3).mean()
        
        # 价格标准化因子
        df['price_volatility_5d'] = df['close'].pct_change().rolling(5).std()
        
        # 开盘跳空因子
        df['opening_gap'] = (df['open'] / df['pre_close'] - 1)
        
        # 1. 最近n天涨幅
        df['price_change_5d'] = df['close'].pct_change(5)  # 5天涨幅
        df['price_change_7d'] = df['close'].pct_change(7)  # 7天涨幅
        df['price_change_10d'] = df['close'].pct_change(10)  # 10天涨幅
        
        # 2. 最近3天是否破15日高点
        # 计算15日最高点（不包括最近3天）
        df['15d_high'] = df['high'].rolling(15).max().shift(3)
        # 检查最近3天最高价是否突破15日高点
        # 使用fillna(0)处理NaN值
        df['break_15d_high'] = (df['high'].rolling(3).max() > df['15d_high']).fillna(0).astype(int)
        
        # 3. 计算涨停板（近似涨停：收盘价 >= 前收盘价 * 1.097）
        df['limit_up'] = (df['close'] >= df['pre_close'] * 1.097).astype(int)
        # 4. 计算跌停板（近似跌停：收盘价 <= 前收盘价 * 0.903）
        df['limit_down'] = (df['close'] <= df['pre_close'] * 0.903).astype(int)
        # 5. 计算一字板（开盘即涨停且全天无波动）
        df['one_word_limit'] = ((df['open'] >= df['pre_close'] * 1.097) & 
                               (df['low'] == df['high'])).astype(int)
        
        # 6. 最近n天涨停数 - 使用fillna(0)处理NaN值
        df['limit_up_count_5d'] = df['limit_up'].rolling(5).sum().fillna(0)
        df['limit_up_count_7d'] = df['limit_up'].rolling(7).sum().fillna(0)
        df['limit_up_count_10d'] = df['limit_up'].rolling(10).sum().fillna(0)
        
        # 7. 最近n天跌停数 - 使用fillna(0)处理NaN值
        df['limit_down_count_5d'] = df['limit_down'].rolling(5).sum().fillna(0)
        df['limit_down_count_7d'] = df['limit_down'].rolling(7).sum().fillna(0)
        df['limit_down_count_10d'] = df['limit_down'].rolling(10).sum().fillna(0)
        
        # 8. 最近n天一字板数 - 使用fillna(0)处理NaN值
        df['one_word_count_5d'] = df['one_word_limit'].rolling(5).sum().fillna(0)
        df['one_word_count_7d'] = df['one_word_limit'].rolling(7).sum().fillna(0)
        df['one_word_count_10d'] = df['one_word_limit'].rolling(10).sum().fillna(0)
        
        # 9. 量比计算（今天对比前n天但不包括今天的量比）
        # 使用过去n天的平均成交量作为基准（不包括当天）

        # 修正量比计算（避免未来函数）
        # df['vol_ma5'] = df['vol'].shift(1).rolling(5).mean()
        df['vol_ma5'] = df['vol'].rolling(5).mean().shift(1)  # 前5天平均成交量（不包括今天）
        df['vol_ma7'] = df['vol'].rolling(7).mean().shift(1)  # 前7天平均成交量（不包括今天）
        df['vol_ma10'] = df['vol'].rolling(10).mean().shift(1)  # 前10天平均成交量（不包括今天）
        
        # 处理NaN值（当分母为0时设为0）
        df['volume_ratio_5d'] = df['vol'] / df['vol_ma5'].replace(0, np.nan).fillna(1)
        df['volume_ratio_7d'] = df['vol'] / df['vol_ma7'].replace(0, np.nan).fillna(1)
        df['volume_ratio_10d'] = df['vol'] / df['vol_ma10'].replace(0, np.nan).fillna(1)
        
        # 10. 最近n天的连续涨停数（最大连续涨停天数）
        # 计算连续涨停序列
        df['limit_streak'] = df['limit_up'] * (df['limit_up'].groupby((df['limit_up'] != df['limit_up'].shift()).cumsum()).cumcount() + 1)
        # 滚动窗口内最大连续涨停数 - 使用fillna(0)处理NaN值
        df['consecutive_limit_5d'] = df['limit_streak'].rolling(5).max().fillna(0)
        df['consecutive_limit_7d'] = df['limit_streak'].rolling(7).max().fillna(0)
        df['consecutive_limit_10d'] = df['limit_streak'].rolling(10).max().fillna(0)
        
        # 11. 今天的振幅
        df['amplitude'] = (df['high'] - df['low']) / df['pre_close'] * 100
        
        # 12. 最近n天的平均多头趋势
        # 定义多头趋势：5日均线 > 10日均线 > 20日均线
        ma_trend = ((df['ma5'] > df['ma10']) & (df['ma10'] > df['ma20']))
        df['ma_trend_3d'] = ma_trend.rolling(3).mean().fillna(0)
        df['ma_trend_5d'] = ma_trend.rolling(5).mean().fillna(0)
        df['ma_trend_7d'] = ma_trend.rolling(7).mean().fillna(0)
        
        # 13. 最近n天的阴阳线数量
        df['is_yang'] = (df['close'] > df['open']).astype(int)  # 阳线
        df['is_yin'] = (df['close'] < df['open']).astype(int)   # 阴线
        
        # 使用fillna(0)处理NaN值
        df['yang_count_5d'] = df['is_yang'].rolling(5).sum().fillna(0)
        df['yang_count_7d'] = df['is_yang'].rolling(7).sum().fillna(0)
        df['yang_count_10d'] = df['is_yang'].rolling(10).sum().fillna(0)
        
        df['yin_count_5d'] = df['is_yin'].rolling(5).sum().fillna(0)
        df['yin_count_7d'] = df['is_yin'].rolling(7).sum().fillna(0)
        df['yin_count_10d'] = df['is_yin'].rolling(10).sum().fillna(0)
    
    def calc_technical_factors(self):
        """技术形态因子计算"""
        df = self.data
        
        # 14. 趋势强度ADX指标（不同周期）
        # 计算不同周期的ADX，并用0填充NaN值
        df['adx_5'] = talib.ADX(df['high'], df['low'], df['close'], timeperiod=5).fillna(0)
        df['adx_7'] = talib.ADX(df['high'], df['low'], df['close'], timeperiod=7).fillna(0)
        df['adx_10'] = talib.ADX(df['high'], df['low'], df['close'], timeperiod=10).fillna(0)
        
        # 15. 布林带宽度指标（不同周期）
        # 5日布林带
        upper5, middle5, lower5 = talib.BBANDS(df['close'], timeperiod=5)
        # df['boll_width_5'] = ((upper5 - lower5) / middle5).fillna(0)
        df['boll_width_5'] = ((upper5 - lower5) / middle5.replace(0, np.nan)).fillna(0)
        
        # 7日布林带
        upper7, middle7, lower7 = talib.BBANDS(df['close'], timeperiod=7)
        df['boll_width_7'] = ((upper7 - lower7) / middle7).fillna(0)
        
        # 10日布林带
        upper10, middle10, lower10 = talib.BBANDS(df['close'], timeperiod=10)
        df['boll_width_10'] = ((upper10 - lower10) / middle10).fillna(0)
        
        # 16. 今天距离最近n天最高点的回调指标
        df['high_5d'] = df['high'].rolling(5).max().fillna(df['close'])
        df['high_7d'] = df['high'].rolling(7).max().fillna(df['close'])
        df['high_10d'] = df['high'].rolling(10).max().fillna(df['close'])
        
        # 避免除以0
        df['pullback_5d'] = (df['high_5d'] - df['close']) / df['high_5d'].replace(0, np.nan).fillna(1)
        df['pullback_7d'] = (df['high_7d'] - df['close']) / df['high_7d'].replace(0, np.nan).fillna(1)
        df['pullback_10d'] = (df['high_10d'] - df['close']) / df['high_10d'].replace(0, np.nan).fillna(1)
    
    def calc_event_factors(self):
        """事件驱动因子计算"""
        df = self.data
        
        # 17. 今天是否一字板（已在基础因子中计算）
        # 18. 今天是否一字板炸板
        # 一字板开盘但收盘未封住涨停
        df['one_word_failed'] = ((df['open'] >= df['pre_close'] * 1.097) & 
                                (df['low'] == df['high']) & 
                                (df['close'] < df['pre_close'] * 1.097)).astype(int)
        
        # 19. 今天是否放量炸板（炸板且量比>2）
        # 炸板：盘中触及涨停但收盘未封住
        df['limit_up_open'] = (df['high'] >= df['pre_close'] * 1.097).astype(int)  # 盘中触及涨停
        df['failed_limit'] = df['limit_up_open'] & (~df['limit_up'])  # 触及涨停但未封住
        
        # 放量炸板：炸板且5天量比>2
        df['volume_failed_limit'] = (df['failed_limit'] & (df['volume_ratio_5d'] > 2)).astype(int)
        
        # 20. 今天是否放量断板
        # 断板：前一天涨停但今天未涨停
        df['limit_up_prev'] = df['limit_up'].shift(1).fillna(0)  # 前一天是否涨停
        df['broken_limit'] = (df['limit_up_prev'] == 1) & (df['limit_up'] == 0)
        
        # 放量断板：断板且量比>2
        df['volume_broken_limit'] = (df['broken_limit'] & (df['volume_ratio_5d'] > 2)).astype(int)
        
        # 21. 今天是否炸板长阴线
        # 炸板长阴线：炸板 + 阴线 + 实体部分>3%
        df['failed_limit_yin'] = (df['failed_limit'] & 
                                 (df['close'] < df['open']) & 
                                 ((df['open'] - df['close']) / df['pre_close'] > 0.03)).astype(int)
        
        # 22. 今天是否炸板跌破5日均线
        df['failed_limit_break_ma5'] = (df['failed_limit'] & (df['close'] < df['ma5'])).astype(int)
        
        # 23. 今天是否放量下跌
        # 放量下跌：阴线 + 量比>2
        df['volume_down'] = ((df['close'] < df['open']) & 
                           (df['volume_ratio_5d'] > 2)).astype(int)
    
    def calc_additional_factors(self):
        """补充核心因子：5-7天趋势 + 短期趋势转变"""
        df = self.data
        
        # ===== 5-7天趋势因子 =====
        # 1. 5日价格通道突破
        df['channel_high_5d'] = df['high'].rolling(5).max()
        df['channel_low_5d'] = df['low'].rolling(5).min()
        df['break_channel_high'] = (df['close'] > df['channel_high_5d'].shift(1)).astype(int)
        df['break_channel_low'] = (df['close'] < df['channel_low_5d'].shift(1)).astype(int)
        
        # 2. 5日动量指标
        df['momentum_5d'] = talib.MOM(df['close'], timeperiod=5)
        
        # 3. 7日相对强弱指标
        df['rsi_7d'] = talib.RSI(df['close'], timeperiod=7)
        
        # 4. 5日累积/派发线
        df['ad_line'] = talib.AD(df['high'], df['low'], df['close'], df['vol'])
        df['ad_5d_change'] = df['ad_line'].diff(5)
        
        # 5. 5日资金流向指标
        df['mfi_5d'] = talib.MFI(df['high'], df['low'], df['close'], df['vol'], timeperiod=5)
        
        # ===== 短期趋势转变因子（1-2天） =====
        # 6. 短期趋势强度变化
        df['trend_strength'] = (df['close'] - df['close'].shift(3)) / df['close'].shift(3)
        df['trend_strength_change'] = df['trend_strength'] - df['trend_strength'].shift(1)
        
        # 7. 量价背离检测
        # 价格新高但成交量未新高
        df['price_new_high'] = (df['close'] == df['close'].rolling(5).max())
        df['volume_not_high'] = (df['vol'] < df['vol'].rolling(5).max().shift(1))
        df['divergence_negative'] = (df['price_new_high'] & df['volume_not_high']).astype(int)
        
        # 价格新低但成交量未新低
        df['price_new_low'] = (df['close'] == df['close'].rolling(5).min())
        df['volume_not_low'] = (df['vol'] > df['vol'].rolling(5).min().shift(1))
        df['divergence_positive'] = (df['price_new_low'] & df['volume_not_low']).astype(int)
        
        # 8. 短期均线交叉
        df['ma3_ma5_cross'] = np.where(
            (df['ma3'] > df['ma5']) & (df['ma3'].shift(1) <= df['ma5'].shift(1)), 1,
            np.where(
                (df['ma3'] < df['ma5']) & (df['ma3'].shift(1) >= df['ma5'].shift(1)), -1, 0
            )
        )
        
        # 9. MACD信号变化
        df['macd'], df['macd_signal'], _ = talib.MACD(df['close'], fastperiod=12, slowperiod=26, signalperiod=9)
        df['macd_cross'] = np.where(
            (df['macd'] > df['macd_signal']) & (df['macd'].shift(1) <= df['macd_signal'].shift(1)), 1,
            np.where(
                (df['macd'] < df['macd_signal']) & (df['macd'].shift(1) >= df['macd_signal'].shift(1)), -1, 0
            )
        )
        
        # 10. 短期波动率变化
        df['atr_5d'] = talib.ATR(df['high'], df['low'], df['close'], timeperiod=5)
        df['atr_change_pct'] = df['atr_5d'].pct_change()
        
        # 11. 短期动能反转
        df['roc_3d'] = talib.ROC(df['close'], timeperiod=3)  # 3日变动率
        df['roc_change'] = df['roc_3d'] - df['roc_3d'].shift(1)
        
        # 12. 支撑/阻力突破
        df['resistance_5d'] = df['high'].rolling(5).max().shift(1)
        df['support_5d'] = df['low'].rolling(5).min().shift(1)
        df['break_resistance'] = (df['close'] > df['resistance_5d']).astype(int)
        df['break_support'] = (df['close'] < df['support_5d']).astype(int)
        
        # 13. 跳空缺口分析
        df['gap_up'] = (df['low'] > df['high'].shift(1)).astype(int)
        df['gap_down'] = (df['high'] < df['low'].shift(1)).astype(int)
        df['gap_filled'] = np.where(
            df['gap_up'], (df['low'] <= df['high'].shift(1)).astype(int),
            np.where(
                df['gap_down'], (df['high'] >= df['low'].shift(1)).astype(int), 0
            )
        )
        
        # 14. 短期超买超卖指标
        df['stoch_k'], df['stoch_d'] = talib.STOCH(
            df['high'], df['low'], df['close'], 
            fastk_period=5, slowk_period=3, slowd_period=3
        )
        df['stoch_overbought'] = (df['stoch_k'] > 80).astype(int)
        df['stoch_oversold'] = (df['stoch_k'] < 20).astype(int)
        
        # 15. 短期趋势加速/减速
        df['price_acceleration'] = df['close'].diff(3) - df['close'].shift(3).diff(3)
        df['volume_acceleration'] = df['vol'].diff(3) - df['vol'].shift(3).diff(3)

    def calc_trend_factors(self):
        """趋势相关因子计算"""
        df = self.data
        
        # 24. 今天是否趋势拐点（多头斜率降，且股价跌）
        # 计算5日均线斜率
        df['ma5_slope'] = df['ma5'].diff().fillna(0)
        # 计算3日均线斜率
        df['ma3_slope'] = df['ma3'].diff().fillna(0)
        
        # 趋势拐点：斜率下降（今日斜率<0且昨日斜率>0）且股价下跌
        # 使用fillna(False)处理NaN值
        df['trend_reversal'] = ((df['ma5_slope'] < 0) & 
                               (df['ma5_slope'].shift(1).fillna(0) > 0) & 
                               (df['close'] < df['close'].shift(1).fillna(df['close']))).astype(int)
        
        # 25. 今天是否趋势回调并计算回调幅度
        # 定义上升趋势：5日线>10日线>20日线
        df['uptrend'] = ((df['ma5'] > df['ma10']) & (df['ma10'] > df['ma20'])).astype(int)
        
        # 趋势回调：上升趋势中收盘价低于5日均线
        df['trend_pullback'] = (df['uptrend'] == 1) & (df['close'] < df['ma5'])
        
        # 回调幅度：(5日均线 - 收盘价) / 5日均线
        # 避免除以0
        df['pullback_ratio'] = np.where(
            df['trend_pullback'],
            (df['ma5'] - df['close']) / df['ma5'].replace(0, np.nan).fillna(0),
            0
        )
        
        # 26. 最近3天是否有横盘后放量涨停
        # 横盘：过去15天振幅<15%
        df['range_15d'] = (df['high'].rolling(15).max() - df['low'].rolling(15).min()) / df['close'].shift(15).replace(0, np.nan).fillna(1)
        df['consolidation'] = (df['range_15d'] < 0.15).fillna(0).astype(int)
        
        # 最近3天内有放量涨停（量比>1.5且涨停）
        # 使用fillna(0)处理NaN值
        df['volume_limit_3d'] = ((df['limit_up'] == 1) & 
                                (df['volume_ratio_5d'] > 1.5)).rolling(3).max().fillna(0).astype(int)
        
        # 横盘后放量涨停
        df['consolidation_volume_limit'] = (df['consolidation'] & df['volume_limit_3d']).astype(int)
        
        # 27. 5日均线的斜率是否出现拐头向下
        # 斜率拐头向下：昨日斜率>0，今日斜率<0
        df['ma5_slope_down'] = ((df['ma5_slope'].shift(1).fillna(0) > 0) & 
                               (df['ma5_slope'] < 0)).astype(int)
        
        # 28. 3日均线的斜率是否出现拐头向下
        df['ma3_slope_down'] = ((df['ma3_slope'].shift(1).fillna(0) > 0) & 
                               (df['ma3_slope'] < 0)).astype(int)
        
        # 29. 检测是否A杀的拐头向下状态
        # A杀特征：连续上涨后突然大幅下跌
        # 连续上涨：至少连续3天收盘价上涨
        df['up_streak'] = (df['close'] > df['close'].shift(1)).rolling(3).sum().fillna(0) == 3
        
        # 大幅下跌：当日跌幅>5%
        df['big_down'] = (df['close'] < df['close'].shift(1) * 0.95).fillna(0).astype(int)
        
        # A杀：连续上涨后出现大幅下跌且5日均线拐头向下
        df['a_kill'] = (df['up_streak'].shift(1).fillna(False) & 
                       df['big_down'] & 
                       df['ma5_slope_down']).astype(int)
        
    def calc_daily_basic_factors(self):
        """基于daily_basic数据的短线因子计算"""
        df = self.data
        
        # 1. 换手率因子
        df['turnover_rate_change_3d'] = df['turnover_rate'].pct_change(3)
        df['turnover_rate_change_5d'] = df['turnover_rate'].pct_change(5)
        df['turnover_rate_deviation'] = (df['turnover_rate'] - df['turnover_rate_5d']) / df['turnover_rate_5d']
        
        # 2. 量比因子
        df['volume_ratio_abnormal'] = np.select(
            [df['volume_ratio'] > 2.5, df['volume_ratio'] < 0.5],
            [1, -1], default=0
        )
        df['volume_ratio_trend'] = df['volume_ratio'].rolling(3).mean().pct_change()
        
        # 3. 估值因子
        df['pe_change_3d'] = df['pe'].pct_change(3).fillna(0)
        df['pe_ttm_change_3d'] = df['pe_ttm'].pct_change(3).fillna(0)
        df['pb_change_3d'] = df['pb'].pct_change(3).fillna(0)
        
        # 4. 市值因子
        df['circ_mv_change_3d'] = df['circ_mv'].pct_change(3).fillna(0)
        df['circ_mv_change_5d'] = df['circ_mv'].pct_change(5).fillna(0)
        df['mv_ratio'] = df['total_mv'] / df['circ_mv'].replace(0, np.nan)
        
        # 5. 自由流通股换手率因子
        df['turnover_rate_f_ma5'] = df['turnover_rate_f'].rolling(5).mean()
        df['turnover_rate_f_spike'] = (df['turnover_rate_f'] > df['turnover_rate_f_ma5'] * 1.8).astype(int)
        
        # 6. 股息率因子
        df['dv_ratio_change_3d'] = df['dv_ratio'].pct_change(3).fillna(0)
        df['dv_ttm_change_3d'] = df['dv_ttm'].pct_change(3).fillna(0)
        
        # 7. 股本变化因子
        df['float_share_change_5d'] = df['float_share'].pct_change(5).fillna(0)
        
        # 8. 量价背离因子
        price_up = (df['close'] > df['close'].shift(1))
        turnover_down = (df['turnover_rate'] < df['turnover_rate'].shift(1))
        df['price_up_turnover_down'] = (price_up & turnover_down).astype(int)
        
        price_down = (df['close'] < df['close'].shift(1))
        turnover_up = (df['turnover_rate'] > df['turnover_rate'].shift(1))
        df['price_down_turnover_up'] = (price_down & turnover_up).astype(int)
        
        # 9. 估值异常因子
        df['negative_pe'] = (df['pe'] < 0).astype(int)
        df['pe_drop_abnormal'] = (df['pe_change_3d'] < -0.3).astype(int)
        
        # 10. 换手率与量比的相关性
        df['high_turnover_low_volume_ratio'] = (
            (df['turnover_rate'] > df['turnover_rate_10d'] * 1.5) & 
            (df['volume_ratio'] < 0.8)
        ).astype(int)
        
        # 11. 小市值高换手组合因子
        df['circ_mv_quantile'] = df['circ_mv'].rolling(60).rank(pct=True)
        df['turnover_rate_quantile'] = df['turnover_rate'].rolling(60).rank(pct=True)
        df['small_cap_high_turnover'] = (
            (df['circ_mv_quantile'] < 0.3) & 
            (df['turnover_rate_quantile'] > 0.7)
        ).astype(int)
        
        # 12. 机构关注度变化
        df['institutional_activity'] = (
            (df['turnover_rate_f'] > df['turnover_rate_f_ma5'] * 1.5) & 
            (df['circ_mv'] > 1000000)
        ).astype(int)
        
        # 13. 市值与成交量的背离
        mv_up = df['circ_mv'] > df['circ_mv'].shift(1)
        vol_down = df['vol'] < df['vol'].shift(1)
        df['mv_up_vol_down'] = (mv_up & vol_down).astype(int)
        
        # 14. 股息率与股价的背离
        dv_up = df['dv_ratio'] > df['dv_ratio'].shift(1)
        price_down = df['close'] < df['close'].shift(1)
        df['dv_up_price_down'] = (dv_up & price_down).astype(int)
        
        # 15. 换手率与市值背离
        turnover_up = df['turnover_rate'] > df['turnover_rate'].shift(1)
        mv_down = df['circ_mv'] < df['circ_mv'].shift(1)
        df['turnover_up_mv_down'] = (turnover_up & mv_down).astype(int)

# ===================== 执行与结果 =====================
# 获取数据 (示例：贵州茅台)
stock_data = prepare_data(ts_code='600519.SH', start_date='20230101', end_date='20231231')

# 生成因子
factor_engine = FactorGenerator(stock_data)
factor_df = factor_engine.data

# 展示最新10个交易日的因子
print(f"因子数量: {len(factor_df.columns)}个")
print(factor_df[['trade_date', 'close', 
                 'price_change_5d', 'price_change_7d', 'price_change_10d',
                 'break_15d_high', 'limit_up_count_5d', 'limit_down_count_5d',
                 'one_word_count_5d', 'volume_ratio_5d', 'consecutive_limit_5d',
                 'amplitude', 'ma_trend_5d', 'yang_count_5d', 'yin_count_5d',
                 'adx_5', 'boll_width_5', 'pullback_5d',
                 'one_word_limit', 'one_word_failed', 'volume_failed_limit',
                 'volume_broken_limit', 'failed_limit_yin', 'failed_limit_break_ma5',
                 'volume_down', 'trend_reversal', 'trend_pullback', 'pullback_ratio',
                 'consolidation_volume_limit', 'ma5_slope_down', 'ma3_slope_down', 'a_kill']].tail(10))

# 保存结果
# factor_df.to_csv('simplified_trading_factors.csv', index=False)

因子数量: 91个
    trade_date    close  price_change_5d  price_change_7d  price_change_10d  \
232   20231218  1662.00        -0.021201        -0.022353         -0.050671   
233   20231219  1675.00        -0.020462        -0.017549         -0.019895   
234   20231220  1649.79        -0.006151        -0.028392         -0.031245   
235   20231221  1670.00         0.020845        -0.023386         -0.017647   
236   20231222  1670.65         0.015593         0.006416         -0.020101   
237   20231225  1672.00         0.006017         0.022067         -0.015312   
238   20231226  1670.00        -0.002985         0.015198         -0.023386   
239   20231227  1667.06         0.010468         0.003045          0.004253   
240   20231228  1724.99         0.032928         0.029845          0.054459   
241   20231229  1726.00         0.033131         0.046194          0.049240   

     break_15d_high  limit_up_count_5d  limit_down_count_5d  \
232               0                0.0                  0

C:\Users\Administrator\AppData\Local\Temp\ipykernel_70504\1158978684.py:272: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['a_kill'] = (df['up_streak'].shift(1).fillna(False) &


In [22]:
import numpy as np
from tqdm import tqdm
from datetime import datetime, timedelta

# 定义基础列名
base_columns = ['ts_code', 'trade_date', 'open', 'high', 'low', 'close', 'pre_close', 
                'change', 'pct_chg', 'vol', 'amount', 'ma5', 'ma10', 'ma20']

# 获取因子列名的函数
def get_factor_columns():
    """通过示例股票获取所有因子列名"""
    try:
        example_stock = '000001.SZ'
        example_data = prepare_data(ts_code=example_stock, start_date='20230101', end_date='20230131')
        if not example_data.empty:
            factor_engine = FactorGenerator(example_data)
            factor_df = factor_engine.data
            factor_columns = [col for col in factor_df.columns if col not in base_columns]
            return factor_columns
        else:
            print("Warning: Failed to get example data for factor columns.")
            return []
    except Exception as e:
        print(f"Error getting factor columns: {str(e)}")
        return []

# 获取所有因子列名
factor_columns = get_factor_columns()
if not factor_columns:
    print("Error: No factor columns found. Please check the data preparation.")
    exit()
print(f"Found {len(factor_columns)} factor columns")

# 为combined_df添加前一个交易日的因子列
for col in factor_columns:
    combined_df[f'prev_{col}'] = np.nan

# 按股票代码分组处理
grouped = combined_df.groupby('stock_code')

# 使用tqdm显示进度条
for stock_code, group in tqdm(grouped, desc="Processing stocks"):
    # 获取该股票在combined_df中的所有日期
    date_keys = group['date_key'].unique()
    
    # 找到最小和最大日期（格式为 %Y-%m-%d）
    min_date_str = min(date_keys)
    max_date_str = max(date_keys)
    
    try:
        # 转换日期格式 %Y-%m-%d -> %Y%m%d
        min_date_dt = datetime.strptime(min_date_str, '%Y-%m-%d')
        max_date_dt = datetime.strptime(max_date_str, '%Y-%m-%d')
        
        # 计算需要的数据范围（前推100天确保有足够历史数据）
        start_date_dt = min_date_dt - timedelta(days=100)
        start_date = start_date_dt.strftime('%Y%m%d')
        end_date = max_date_dt.strftime('%Y%m%d')
    except ValueError as e:
        print(f"Invalid date format for stock {stock_code}: {min_date_str} or {max_date_str}. Error: {str(e)}")
        continue
    
    try:
        # 获取股票数据
        stock_df = prepare_data(ts_code=stock_code, start_date=start_date, end_date=end_date)
        
        if stock_df.empty:
            print(f"Warning: No data for {stock_code} from {start_date} to {end_date}")
            continue
            
        # 计算因子
        factor_engine = FactorGenerator(stock_df)
        factor_df = factor_engine.data
        
        # 确保trade_date是整数类型（%Y%m%d格式）
        if 'trade_date' not in factor_df.columns:
            print(f"Error: 'trade_date' column missing for {stock_code}")
            continue
            
        try:
            factor_df['trade_date'] = factor_df['trade_date'].astype(int)
        except Exception as e:
            print(f"Error converting trade_date to int for {stock_code}: {str(e)}")
            continue
        
        # 创建因子数据的副本并设置索引
        factor_df_indexed = factor_df.set_index('trade_date').copy()
        
        # 处理该组中的每一行
        for idx, row in group.iterrows():
            date_key = row['date_key']  # 格式为 %Y-%m-%d
            
            try:
                # 获取前一个交易日（格式为 %Y-%m-%d）
                prev_date_str = date.get_previous_trade_date(date_key)
                
                # 将前一个交易日转换为整数格式（%Y%m%d）
                prev_date_dt = datetime.strptime(prev_date_str, '%Y-%m-%d')
                prev_date_int = int(prev_date_dt.strftime('%Y%m%d'))
                
                # 在因子数据中查找前一个交易日的因子
                if prev_date_int in factor_df_indexed.index:
                    prev_factors = factor_df_indexed.loc[prev_date_int]
                    
                    # 添加所有因子列的值
                    for col in factor_columns:
                        if col in prev_factors:
                            combined_df.at[idx, f'prev_{col}'] = prev_factors[col]
                        else:
                            # 如果因子列不存在，保留NaN值
                            pass
                else:
                    # 如果前一个交易日不在因子数据中，跳过
                    pass
                    
            except Exception as e:
                print(f"Error processing row {idx} for {stock_code} on {date_key}: {str(e)}")
    
    except Exception as e:
        print(f"Error processing {stock_code}: {str(e)}")

print("Feature extraction completed!")

C:\Users\Administrator\AppData\Local\Temp\ipykernel_70504\1158978684.py:272: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['a_kill'] = (df['up_streak'].shift(1).fillna(False) &


Found 77 factor columns


Processing stocks:   0%|          | 0/3068 [00:00<?, ?it/s]C:\Users\Administrator\AppData\Local\Temp\ipykernel_70504\1158978684.py:272: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['a_kill'] = (df['up_streak'].shift(1).fillna(False) &
C:\Users\Administrator\AppData\Local\Temp\ipykernel_70504\3873339599.py:108: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  combined_df.at[idx, f'prev_{col}'] = prev_factors[col]
C:\Users\Administrator\AppData\Local\Temp\ipykernel_70504\3873339599.py:108: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future ve

Feature extraction completed!


In [23]:
# ... (前面的特征提取代码) ...

print("Feature extraction completed!")

# ===================== 缺失值填充 =====================
print("Filling missing values...")

# 获取所有新增的prev_列
prev_columns = [col for col in combined_df.columns if col.startswith('prev_')]

# 根据因子类型定义填充策略
for col in prev_columns:
    # 获取原始因子列名
    original_col = col.replace('prev_', '')
    
    # 根据因子类型使用不同的填充策略
    if 'count' in original_col or 'streak' in original_col or 'consecutive' in original_col:
        # 计数类因子：填充为0
        combined_df[col] = combined_df[col].fillna(0)
    elif 'ratio' in original_col or 'change' in original_col or 'pct' in original_col:
        # 比例/变化类因子：填充为0.0
        combined_df[col] = combined_df[col].fillna(0.0)
    elif 'limit' in original_col or 'failed' in original_col or 'broken' in original_col or 'trend' in original_col or 'reversal' in original_col:
        # 事件标志类因子：填充为0
        combined_df[col] = combined_df[col].fillna(0)
    elif 'adx' in original_col or 'boll' in original_col or 'ma' in original_col:
        # 技术指标类因子：填充为0.0
        combined_df[col] = combined_df[col].fillna(0.0)
    elif 'pullback' in original_col or 'amplitude' in original_col:
        # 回调/振幅类因子：填充为0.0
        combined_df[col] = combined_df[col].fillna(0.0)
    elif 'yang' in original_col or 'yin' in original_col:
        # 阴阳线计数类因子：填充为0
        combined_df[col] = combined_df[col].fillna(0)
    else:
        # 其他未知类型的因子：使用该列的均值填充
        col_mean = combined_df[col].mean()
        combined_df[col] = combined_df[col].fillna(col_mean)
        print(f"Filled {col} with mean value: {col_mean:.4f}")

print("Missing values filled successfully!")

Feature extraction completed!
Filling missing values...
Filled prev_15d_high with mean value: 14.9600
Filled prev_break_15d_high with mean value: 0.3925
Filled prev_high_5d with mean value: 14.8002
Filled prev_high_7d with mean value: 14.9387
Filled prev_high_10d with mean value: 15.0995
Filled prev_volume_down with mean value: 0.0482
Filled prev_range_15d with mean value: 0.2641
Filled prev_consolidation with mean value: 0.3065
Filled prev_big_down with mean value: 0.0837
Filled prev_a_kill with mean value: 0.0018
Missing values filled successfully!


In [42]:
combined_df

,id,date_key,stock_code,stock_name,block_category,block_codes,industry_code,max_block_category_rank,max_block_code_rank,max_industry_code_rank,...,prev_pullback_ratio,prev_range_15d,prev_consolidation,prev_volume_limit_3d,prev_consolidation_volume_limit,prev_ma5_slope_down,prev_ma3_slope_down,prev_up_streak,prev_big_down,prev_a_kill
0,1630,2024-09-02,000001.SZ,平安银行,"000003.BKDL,000038.BKDL,000023.BKDL,000037.BKDL","885694.DDBK,885867.DDBK,885745.DDBK,883300.DDB...",,7,2,-1,...,0.000000,0.067729,1.0,0.0,0.0,0.0,0.0,False,0.0,0.0
1,1606,2024-09-02,000008.SZ,神州高铁,"000012.BKDL,000038.BKDL,000023.BKDL,000035.BKD...","886021.DDBK,885867.DDBK,885517.DDBK,883437.DDB...",,2,1,-1,...,0.000000,0.134615,1.0,0.0,0.0,0.0,0.0,False,0.0,0.0
2,1971,2024-09-02,000026.SZ,飞亚达,"000023.BKDL,000014.BKDL,000010.BKDL,000017.BKDL","885420.DDBK,886021.DDBK,886072.DDBK,885595.DDB...",,10,21,-1,...,0.000000,0.091314,1.0,0.0,0.0,0.0,0.0,True,0.0,0.0
3,16594,2024-09-02,000058.SZ,深 赛 格,"000012.BKDL,000038.BKDL,000023.BKDL,000035.BKD...","886021.DDBK,885574.DDBK,885729.DDBK,885697.DDB...",,2,1,-1,...,0.000000,0.175407,0.0,0.0,0.0,0.0,0.0,False,0.0,0.0
4,9542,2024-09-02,000065.SZ,北方国际,"000008.BKDL,000038.BKDL,000026.BKDL,000035.BKD...","886045.DDBK,885996.DDBK,885840.DDBK,886021.DDB...",,6,2,-1,...,0.000000,0.116156,1.0,0.0,0.0,0.0,0.0,False,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86823,53299,2025-07-25,605358.SH,立昂微,"000038.BKDL,000035.BKDL,000031.BKDL,000027.BKD...","885908.DDBK,885929.DDBK,885545.DDBK,885461.DDB...",,5,11,-1,...,0.000000,0.047619,1.0,0.0,0.0,0.0,0.0,False,0.0,0.0
86824,51773,2025-07-25,605366.SH,宏柏新材,"000036.BKDL,000023.BKDL,000035.BKDL,000037.BKD...","885520.DDBK,885912.DDBK,886089.DDBK,885958.DDB...",,1,14,-1,...,0.000000,0.332808,0.0,0.0,0.0,0.0,0.0,False,0.0,0.0
86825,51703,2025-07-25,605488.SH,福莱新材,"000036.BKDL,000023.BKDL,000006.BKDL,000029.BKD...","885869.DDBK,885806.DDBK,885800.DDBK,885929.DDB...",,1,11,-1,...,0.010593,0.142578,1.0,0.0,0.0,0.0,0.0,False,0.0,0.0
86826,52505,2025-07-25,605500.SH,森林包装,"000023.BKDL,000004.BKDL","885924.DDBK,886079.DDBK",,1,3,-1,...,0.000000,0.794903,0.0,0.0,0.0,0.0,0.0,False,1.0,0.0


In [24]:
# 定义要过滤掉的日期列表
filter_dates = ['2025-07-25', '2025-07-24', '2025-07-23', '2025-07-22']

# 使用布尔索引过滤 DataFrame
combined_df = combined_df[~combined_df['date_key'].isin(filter_dates)]

# 重置索引
combined_df = combined_df.reset_index(drop=True)

combined_df

,id,date_key,stock_code,stock_name,block_category,block_codes,industry_code,max_block_category_rank,max_block_code_rank,max_industry_code_rank,...,prev_pullback_ratio,prev_range_15d,prev_consolidation,prev_volume_limit_3d,prev_consolidation_volume_limit,prev_ma5_slope_down,prev_ma3_slope_down,prev_up_streak,prev_big_down,prev_a_kill
0,1630,2024-09-02,000001.SZ,平安银行,"000003.BKDL,000038.BKDL,000023.BKDL,000037.BKDL","885694.DDBK,885867.DDBK,885745.DDBK,883300.DDB...",,7,2,-1,...,0.000000,0.067729,1.0,0.0,0.0,0.0,0.0,False,0.0,0.0
1,16559,2024-09-02,000008.SZ,神州高铁,"000012.BKDL,000038.BKDL,000023.BKDL,000035.BKD...","886021.DDBK,885867.DDBK,885517.DDBK,883437.DDB...",,2,1,-1,...,0.000000,0.134615,1.0,0.0,0.0,0.0,0.0,False,0.0,0.0
2,16304,2024-09-02,000026.SZ,飞亚达,"000023.BKDL,000014.BKDL,000010.BKDL,000017.BKDL","885420.DDBK,886021.DDBK,886072.DDBK,885595.DDB...",,10,21,-1,...,0.000000,0.091314,1.0,0.0,0.0,0.0,0.0,True,0.0,0.0
3,16594,2024-09-02,000058.SZ,深 赛 格,"000012.BKDL,000038.BKDL,000023.BKDL,000035.BKD...","886021.DDBK,885574.DDBK,885729.DDBK,885697.DDB...",,2,1,-1,...,0.000000,0.175407,0.0,0.0,0.0,0.0,0.0,False,0.0,0.0
4,9542,2024-09-02,000065.SZ,北方国际,"000008.BKDL,000038.BKDL,000026.BKDL,000035.BKD...","886045.DDBK,885996.DDBK,885840.DDBK,886021.DDB...",,6,2,-1,...,0.000000,0.116156,1.0,0.0,0.0,0.0,0.0,False,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84761,34576,2025-07-21,605287.SH,德才股份,"000012.BKDL,000023.BKDL,000035.BKDL,000007.BKDL","885991.DDBK,886089.DDBK,885709.DDBK,885729.DDB...",,3,14,-1,...,0.015337,0.129055,1.0,0.0,0.0,1.0,1.0,False,0.0,0.0
84762,38011,2025-07-21,605336.SH,帅丰电器,"000023.BKDL,000010.BKDL","885420.DDBK,886072.DDBK,885543.DDBK,885784.DDBK",,3,29,-1,...,0.000000,0.288950,0.0,0.0,0.0,0.0,0.0,False,0.0,0.0
84763,34724,2025-07-21,605366.SH,宏柏新材,"000036.BKDL,000023.BKDL,000035.BKDL,000037.BKD...","885520.DDBK,885912.DDBK,886089.DDBK,885958.DDB...",,3,6,-1,...,0.076621,0.355738,0.0,0.0,0.0,0.0,0.0,False,0.0,0.0
84764,37130,2025-07-21,605399.SH,晨光新材,"000036.BKDL,000035.BKDL,000037.BKDL,000022.BKD...","885520.DDBK,885912.DDBK,883303.DDBK,885929.DDB...",,1,2,-1,...,0.062458,0.508434,0.0,0.0,0.0,1.0,0.0,False,0.0,0.0


In [25]:
import numpy as np
import pandas as pd
from tqdm import tqdm

# 定义要处理的字段列表
target_fields = ['cjs', 'jsjl', 'xcjw', 'jssb']

# 定义 n 的取值列表并排序（从小到大）
n_list = sorted([5, 10, 15, 20, 25, 30, 35])
max_n = max(n_list)  # 最大窗口值

# 为每个字段和每个 n 创建新列
for field in target_fields:
    for n in n_list:
        # 添加前 n 日非零均值列
        combined_df[f'{field}_prev_{n}d_mean'] = 0.0
        # 添加前 n 日总值列
        combined_df[f'{field}_prev_{n}d_sum'] = 0.0

# 按股票代码分组处理
grouped = combined_df.groupby('stock_code')

# 使用 tqdm 显示进度条
for stock_code, group in tqdm(grouped, desc="Processing stocks"):
    # 获取该股票的所有日期（按日期排序）
    sorted_dates = group['date_key'].sort_values().unique()
    
    # 创建临时列表存储索引和结果
    index_list = []
    result_dict = {f'{field}_prev_{n}d_mean': [] for field in target_fields for n in n_list}
    result_dict.update({f'{field}_prev_{n}d_sum': [] for field in target_fields for n in n_list})
    
    # 处理每个日期
    for date_key in sorted_dates:
        # 获取当前日期在分组中的行索引
        current_row = group[group['date_key'] == date_key]
        if not current_row.empty:
            idx = current_row.index[0]
            index_list.append(idx)
            
            # 一次性获取前 max_n 个交易日的日期
            prev_dates = get_previous_trade_date_n(date_key, max_n)
            
            # 获取这些日期的数据行
            prev_data = group[group['date_key'].isin(prev_dates)]
            
            # 计算每个字段在不同窗口下的均值和总值
            for field in target_fields:
                # 为每个 n 值计算
                for n_val in n_list:
                    # 获取前 n_val 个交易日
                    window_dates = prev_dates[:min(len(prev_dates), n_val)]
                    
                    # 获取这些日期在数据框中的行
                    window_data = prev_data[prev_data['date_key'].isin(window_dates)]
                    
                    # 提取字段值，只考虑实际存在的行
                    if not window_data.empty:
                        field_values = window_data[field].dropna()
                    else:
                        field_values = pd.Series(dtype=float)
                    
                    # 计算非零数据的均值（只考虑非零值）
                    non_zero_values = field_values[field_values != 0]
                    if len(non_zero_values) > 0:
                        mean_val = non_zero_values.mean()
                    else:
                        mean_val = 0.0
                    
                    # 计算总值（包括所有值）
                    sum_val = field_values.sum() if not field_values.empty else 0.0
                    
                    # 存储结果
                    result_dict[f'{field}_prev_{n_val}d_mean'].append(mean_val)
                    result_dict[f'{field}_prev_{n_val}d_sum'].append(sum_val)
        else:
            # 如果找不到当前日期的行，跳过
            continue
    
    # 将结果添加回原数据框
    for col_name, values in result_dict.items():
        if len(values) == len(index_list):
            for i, idx in enumerate(index_list):
                combined_df.at[idx, col_name] = values[i]

print("Feature calculation completed!")

Processing stocks: 100%|██████████| 3051/3051 [44:50<00:00,  1.13it/s]  

Feature calculation completed!


In [26]:
# import pandas as pd
# import numpy as np
# from tqdm import tqdm
from datetime import datetime, timedelta

def get_previous_day(date_str):
    """获取日历日前一天的日期（不考虑交易日）"""
    date_obj = datetime.strptime(date_str, '%Y-%m-%d')
    prev_day = date_obj - timedelta(days=1)
    return prev_day.strftime('%Y-%m-%d')

# 添加新列
combined_df['ths_hot_rank'] = 101  # 默认值101表示不在前100名
combined_df['ths_hot_score'] = 0   # 热度分默认0
combined_df['dc_hot_rank'] = 101   # 默认值101表示不在前100名

# 获取所有需要查询的日期
all_dates = combined_df['date_key'].unique()

# 创建缓存字典
ths_cache = {}
dc_cache = {}

# 获取前一日热榜数据并缓存
for date in tqdm(all_dates, desc="Fetching hot data"):
    # 获取日历日前一天
    prev_date = get_previous_day(date)
    
    # 同花顺热榜数据
    if prev_date not in ths_cache:
        try:
            # 转换日期格式为YYYYMMDD
            prev_date_fmt = prev_date.replace('-', '')
            ths_df = pro.ths_hot(trade_date=prev_date_fmt, market='热股', fields='ts_code,ts_name,hot,concept')
            # 使用 empty 属性判断 DataFrame 是否为空
            if ths_df.empty:
                raise Exception("No data returned from THS hot query")
            
            # 添加排名列（行号+1）
            if not ths_df.empty:
                ths_df['rank'] = ths_df.index + 1
            ths_cache[prev_date] = ths_df
        except Exception as e:
            print(f"Error fetching THS hot data for {prev_date}: {str(e)}")
            ths_cache[prev_date] = pd.DataFrame()
    
    # 东方财富热榜数据
    if prev_date not in dc_cache:
        try:
            # 转换日期格式为YYYYMMDD
            prev_date_fmt = prev_date.replace('-', '')
            dc_df = pro.dc_hot(trade_date=prev_date_fmt, market='A股市场', hot_type='人气榜', fields='ts_code,ts_name,rank')
            # 使用 empty 属性判断 DataFrame 是否为空
            if dc_df.empty:
                raise Exception("No data returned from DC hot query")
            dc_cache[prev_date] = dc_df
        except Exception as e:
            print(f"Error fetching DC hot data for {prev_date}: {str(e)}")
            dc_cache[prev_date] = pd.DataFrame()

# 为combined_df添加热榜数据
for idx, row in tqdm(combined_df.iterrows(), total=len(combined_df), desc="Adding hot ranks"):
    date_key = row['date_key']
    stock_code = row['stock_code']
    
    # 获取日历日前一天
    prev_date = get_previous_day(date_key)
    
    # 处理同花顺热榜数据
    if prev_date in ths_cache:
        ths_df = ths_cache[prev_date]
        if not ths_df.empty:
            # 查找股票在同花顺热榜中的位置
            ths_row = ths_df[ths_df['ts_code'] == stock_code]
            if not ths_row.empty:
                combined_df.at[idx, 'ths_hot_rank'] = ths_row['rank'].values[0]
                combined_df.at[idx, 'ths_hot_score'] = ths_row['hot'].values[0]
    
    # 处理东方财富热榜数据
    if prev_date in dc_cache:
        dc_df = dc_cache[prev_date]
        if not dc_df.empty:
            # 查找股票在东方财富热榜中的位置
            dc_row = dc_df[dc_df['ts_code'] == stock_code]
            if not dc_row.empty:
                combined_df.at[idx, 'dc_hot_rank'] = dc_row['rank'].values[0]

print("Hot rank data added successfully!")

Fetching hot data:   0%|          | 0/212 [00:00<?, ?it/s]

Fetching hot data:   5%|▌         | 11/212 [00:02<01:05,  3.06it/s]

Error fetching THS hot data for 2024-09-17: No data returned from THS hot query
Error fetching DC hot data for 2024-09-17: No data returned from DC hot query


Fetching hot data:  75%|███████▌  | 159/212 [00:32<00:07,  7.24it/s]

Error fetching DC hot data for 2025-04-29: No data returned from DC hot query
Error fetching THS hot data for 2025-05-05: No data returned from THS hot query
Error fetching DC hot data for 2025-05-05: No data returned from DC hot query


Fetching hot data:  88%|████████▊ | 186/212 [00:36<00:03,  6.72it/s]

Error fetching DC hot data for 2025-06-11: No data returned from DC hot query
Error fetching DC hot data for 2025-06-12: No data returned from DC hot query


Adding hot ranks: 100%|██████████| 84766/84766 [00:27<00:00, 3043.94it/s]

Hot rank data added successfully!


In [27]:
combined_df

,id,date_key,stock_code,stock_name,block_category,block_codes,industry_code,max_block_category_rank,max_block_code_rank,max_industry_code_rank,...,jssb_prev_20d_sum,jssb_prev_25d_mean,jssb_prev_25d_sum,jssb_prev_30d_mean,jssb_prev_30d_sum,jssb_prev_35d_mean,jssb_prev_35d_sum,ths_hot_rank,ths_hot_score,dc_hot_rank
0,1630,2024-09-02,000001.SZ,平安银行,"000003.BKDL,000038.BKDL,000023.BKDL,000037.BKDL","885694.DDBK,885867.DDBK,885745.DDBK,883300.DDB...",,7,2,-1,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,101,0,101
1,16559,2024-09-02,000008.SZ,神州高铁,"000012.BKDL,000038.BKDL,000023.BKDL,000035.BKD...","886021.DDBK,885867.DDBK,885517.DDBK,883437.DDB...",,2,1,-1,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,101,0,101
2,16304,2024-09-02,000026.SZ,飞亚达,"000023.BKDL,000014.BKDL,000010.BKDL,000017.BKDL","885420.DDBK,886021.DDBK,886072.DDBK,885595.DDB...",,10,21,-1,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,101,0,101
3,16594,2024-09-02,000058.SZ,深 赛 格,"000012.BKDL,000038.BKDL,000023.BKDL,000035.BKD...","886021.DDBK,885574.DDBK,885729.DDBK,885697.DDB...",,2,1,-1,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,101,0,101
4,9542,2024-09-02,000065.SZ,北方国际,"000008.BKDL,000038.BKDL,000026.BKDL,000035.BKD...","886045.DDBK,885996.DDBK,885840.DDBK,886021.DDB...",,6,2,-1,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,101,0,101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84761,34576,2025-07-21,605287.SH,德才股份,"000012.BKDL,000023.BKDL,000035.BKDL,000007.BKDL","885991.DDBK,886089.DDBK,885709.DDBK,885729.DDB...",,3,14,-1,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,101,0,101
84762,38011,2025-07-21,605336.SH,帅丰电器,"000023.BKDL,000010.BKDL","885420.DDBK,886072.DDBK,885543.DDBK,885784.DDBK",,3,29,-1,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,101,0,101
84763,34724,2025-07-21,605366.SH,宏柏新材,"000036.BKDL,000023.BKDL,000035.BKDL,000037.BKD...","885520.DDBK,885912.DDBK,886089.DDBK,885958.DDB...",,3,6,-1,...,0.0,0.0,0.0,0.0,0.0,0.06,0.06,101,0,101
84764,37130,2025-07-21,605399.SH,晨光新材,"000036.BKDL,000035.BKDL,000037.BKDL,000022.BKD...","885520.DDBK,885912.DDBK,883303.DDBK,885929.DDB...",,1,2,-1,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,101,0,101


In [28]:
# from datetime import datetime, timedelta
# from tqdm import tqdm
# import time
# import tushare as ts  # 导入 Tushare

# # 初始化 Tushare
# pro = ts.pro_api('your_tushare_token')  # 替换为你的 Tushare token
ts.set_token('c4dc1a3297388071b02491bc968246f502cccd5e761ee84655a0adc0')
pro = ts.pro_api()

def download_index_data(stock_codes, start_date, end_date):
    """
    下载指数数据
    """
    # print(f"开始下载指数数据: {start_date} 至 {end_date}")
    
    # 下载数据
    download_tasks = []
    for code in stock_codes:
        task_id = xtdata.download_history_data(
            stock_code=code,
            period="1d",
            start_time=start_date,  # 使用YYYYMMDD格式
            end_time=end_date       # 使用YYYYMMDD格式
        )
        download_tasks.append(task_id)
    
    # print("指数数据下载完成")

def download_stock_data(date):
    """
    下载个股数据
    """
    # print(f"开始下载个股数据: {date}")
    
    # 获取沪深A股所有股票代码
    all_stocks = xtdata.get_stock_list_in_sector('沪深A股')
    
    # 下载数据
    task_id = xtdata.download_history_data(
        stock_code=all_stocks,
        period="1d",
        start_time=date,  # 使用YYYYMMDD格式
        end_time=date    # 使用YYYYMMDD格式
    )
    
    # print("个股数据下载完成")

def get_market_factors(today, prev_date):
    """
    生成当日大盘因子（开盘前可用）
    
    参数:
    today (str): 当前日期，格式为 'YYYYMMDD'
    prev_date (str): 前一交易日，格式为 'YYYYMMDD'
    """
    # 定义核心指数代码
    indices = {
        "sh": "000001.SH",        # 上证指数
        "sz": "399001.SZ",        # 深证成指
        "cyb": "399006.SZ",       # 创业板指
        "kcb50": "000688.SH"      # 科创50
    }
    index_codes = list(indices.values())
    
    # 存储因子结果的字典
    factors = {}
    
    # 0. 下载所需数据
    try:
        # 下载指数数据
        download_index_data(index_codes, prev_date, prev_date)
        download_index_data(index_codes, today, today)
        
        # 下载个股数据（用于市场广度指标）
        # download_stock_data(prev_date)
    except Exception as e:
        print(f"数据下载失败: {e}")
    
    # 1. 获取前一交易日收盘数据
    df_prev = pd.DataFrame()
    try:
        # 批量获取所有指数前一交易日数据
        prev_data = xtdata.get_market_data_ex(
            field_list=['close'],
            stock_list=index_codes,
            period='1d',
            start_time=prev_date,  # 使用YYYYMMDD格式
            end_time=prev_date,    # 使用YYYYMMDD格式
            fill_data=True
        )
        
        for code, data in prev_data.items():
            if not data.empty:
                # 提取收盘价
                close = data['close'].iloc[0]
                # 添加到DataFrame
                df_prev = pd.concat([df_prev, pd.DataFrame({
                    'ts_code': [code],
                    'close': [close]
                })])
    except Exception as e:
        print(f"获取前一交易日数据失败: {e}")
    
    # 2. 获取当日开盘数据
    df_today = pd.DataFrame()
    try:
        # 批量获取所有指数当日数据
        today_data = xtdata.get_market_data_ex(
            field_list=['open'],
            stock_list=index_codes,
            period='1d',
            start_time=today,  # 使用YYYYMMDD格式
            end_time=today,    # 使用YYYYMMDD格式
            fill_data=True
        )
        
        for code, data in today_data.items():
            if not data.empty:
                # 提取开盘价
                open_price = data['open'].iloc[0]
                # 添加到DataFrame
                df_today = pd.concat([df_today, pd.DataFrame({
                    'ts_code': [code],
                    'open': [open_price]
                })])
    except Exception as e:
        print(f"获取当日数据失败: {e}")
    
    # 3. 计算开盘强度因子
    for prefix, code in indices.items():
        # 获取前一交易日收盘价
        prev_data = df_prev[df_prev['ts_code'] == code]
        if not prev_data.empty:
            prev_close = prev_data['close'].values[0]
        else:
            print(f"警告: 未获取到{code}在{prev_date}的数据")
            prev_close = np.nan
        
        # 获取当日开盘价
        today_data = df_today[df_today['ts_code'] == code]
        if not today_data.empty:
            today_open = today_data['open'].values[0]
        else:
            print(f"警告: 未获取到{code}在{today}的开盘数据")
            today_open = np.nan
        
        # 计算开盘涨跌幅
        if not np.isnan(prev_close) and not np.isnan(today_open) and prev_close > 0:
            open_pct = (today_open / prev_close - 1) * 100
            factors[f"{prefix}_open_pct"] = open_pct
        else:
            factors[f"{prefix}_open_pct"] = np.nan
        
        # 计算相对前高缺口
        if not np.isnan(today_open):
            # 获取前5日历史数据
            prev_dt = datetime.strptime(prev_date, '%Y%m%d')
            start_dt = prev_dt - timedelta(days=7)
            start_date = start_dt.strftime("%Y%m%d")
            
            try:
                # 下载历史数据
                download_index_data([code], start_date, prev_date)
                
                # 获取历史数据
                hist_data = xtdata.get_market_data_ex(
                    field_list=['high'],
                    stock_list=[code],
                    period='1d',
                    start_time=start_date,  # 使用YYYYMMDD格式
                    end_time=prev_date,     # 使用YYYYMMDD格式
                    fill_data=True
                )
                
                if code in hist_data and not hist_data[code].empty:
                    high_5d = hist_data[code]['high'].max()
                    if not np.isnan(high_5d) and high_5d > 0:
                        gap_pct = (today_open - high_5d) / high_5d * 100
                        factors[f"{prefix}_gap_vs_high"] = gap_pct
                    else:
                        factors[f"{prefix}_gap_vs_high"] = np.nan
                else:
                    factors[f"{prefix}_gap_vs_high"] = np.nan
            except Exception as e:
                print(f"获取{code}历史数据失败: {e}")
                factors[f"{prefix}_gap_vs_high"] = np.nan
        else:
            factors[f"{prefix}_gap_vs_high"] = np.nan
    
    # 4. 计算历史趋势因子
    for prefix, code in indices.items():
        # 获取历史数据（不包括当天）
        prev_dt = datetime.strptime(prev_date, '%Y%m%d')
        start_dt = prev_dt - timedelta(days=10)
        start_date = start_dt.strftime("%Y%m%d")
        
        try:
            # 下载历史数据
            download_index_data([code], start_date, prev_date)
            
            # 获取历史数据
            hist_data = xtdata.get_market_data_ex(
                field_list=['close', 'amount'],
                stock_list=[code],
                period='1d',
                start_time=start_date,  # 使用YYYYMMDD格式
                end_time=prev_date,     # 使用YYYYMMDD格式
                fill_data=True
            )
            
            if code not in hist_data or hist_data[code].empty:
                print(f"警告: {code}历史数据不足")
                factors[f"{prefix}_return_3d"] = np.nan
                factors[f"{prefix}_volatility_5d"] = np.nan
                factors[f"{prefix}_volume_ratio"] = np.nan
                continue
                
            df_hist = hist_data[code].sort_index()  # 按时间排序
            close_vals = df_hist['close'].values
            amount_vals = df_hist['amount'].values
            
            # 3日收益率 (使用历史收盘价)
            if len(close_vals) >= 4:
                factors[f"{prefix}_return_3d"] = (close_vals[-1] / close_vals[-4] - 1) * 100
            else:
                factors[f"{prefix}_return_3d"] = np.nan
            
            # 5日波动率 (基于历史数据)
            if len(close_vals) >= 5:
                factors[f"{prefix}_volatility_5d"] = np.std(close_vals[-5:]) / np.mean(close_vals[-5:])
            else:
                factors[f"{prefix}_volatility_5d"] = np.nan
            
            # 量能变化率 (基于历史成交额)
            if len(amount_vals) >= 5:
                factors[f"{prefix}_volume_ratio"] = amount_vals[-1] / np.mean(amount_vals[-5:-1])
            else:
                factors[f"{prefix}_volume_ratio"] = np.nan
        except Exception as e:
            print(f"获取{code}历史数据失败: {e}")
            factors[f"{prefix}_return_3d"] = np.nan
            factors[f"{prefix}_volatility_5d"] = np.nan
            factors[f"{prefix}_volume_ratio"] = np.nan
    
    # 5. 获取富时中国A50指数数据（保持原方式）
    try:
        # 获取A50最新数据（通常是隔夜数据）
        df_a50 = pro.index_global(ts_code="XIN9", trade_date=prev_date)
        if not df_a50.empty:
            factors["a50_change"] = df_a50['pct_chg'].values[0]
        else:
            factors["a50_change"] = np.nan
    except Exception as e:
        print(f"获取富时A50数据失败: {e}")
        factors["a50_change"] = np.nan
    
    # 6. 获取市场广度指标（使用 Tushare）
    try:
        # 使用 Tushare 获取前一交易日所有个股数据
        df_stocks = pro.daily(trade_date=prev_date)
        
        if not df_stocks.empty:
            # 计算上涨和下跌家数
            up_count = (df_stocks['pct_chg'] > 0).sum()
            down_count = (df_stocks['pct_chg'] < 0).sum()
            total = up_count + down_count
            
            # 计算涨跌比
            if total > 0:
                factors["advance_decline_ratio"] = up_count / total
            else:
                factors["advance_decline_ratio"] = 0.5
        else:
            factors["advance_decline_ratio"] = np.nan
    except Exception as e:
        print(f"获取市场广度指标失败: {e}")
        factors["advance_decline_ratio"] = np.nan
    
    # 转换为DataFrame并返回
    return pd.DataFrame([factors], index=[today])

def add_market_factors(combined_df, get_previous_trade_date):
    """
    为combined_df添加大盘因子列
    
    参数:
    combined_df (pd.DataFrame): 包含stock_code和date_key的DataFrame
    get_previous_trade_date (function): 获取前一交易日的函数
    
    返回:
    pd.DataFrame: 添加了大盘因子的DataFrame
    """
    # 1. 准备唯一日期列表并初始化因子缓存
    unique_dates = combined_df['date_key'].unique().tolist()
    factors_cache = {}
    
    # 2. 为每个唯一日期计算大盘因子
    for date_str in tqdm(unique_dates, desc="处理大盘因子"):
        # 转换日期格式 (YYYY-MM-DD -> YYYYMMDD)
        today_ymd = date_str.replace("-", "")
        
        # 获取前一交易日
        prev_date = get_previous_trade_date(date_str)
        if not prev_date:
            print(f"未找到{date_str}的前一交易日")
            continue
            
        # 转换格式
        prev_date_ymd = prev_date.replace("-", "")
        
        # 检查缓存
        cache_key = (today_ymd, prev_date_ymd)
        if cache_key in factors_cache:
            continue
            
        try:
            # 获取大盘因子
            factor_df = get_market_factors(today_ymd, prev_date_ymd)
            
            # 转换索引为带横杠的日期格式
            factor_df.index = factor_df.index.map(lambda x: f"{x[:4]}-{x[4:6]}-{x[6:]}")
            factors_cache[cache_key] = factor_df
        except Exception as e:
            print(f"计算{date_str}的因子时出错: {e}")
            # 创建空因子行
            # 获取列名（如果可能）
            try:
                # 使用虚拟日期创建空因子
                empty_factors = {
                    "sh_open_pct": np.nan, "sz_open_pct": np.nan, "cyb_open_pct": np.nan, "kcb50_open_pct": np.nan,
                    "sh_gap_vs_high": np.nan, "sz_gap_vs_high": np.nan, "cyb_gap_vs_high": np.nan, "kcb50_gap_vs_high": np.nan,
                    "sh_return_3d": np.nan, "sz_return_3d": np.nan, "cyb_return_3d": np.nan, "kcb50_return_3d": np.nan,
                    "sh_volatility_5d": np.nan, "sz_volatility_5d": np.nan, "cyb_volatility_5d": np.nan, "kcb50_volatility_5d": np.nan,
                    "sh_volume_ratio": np.nan, "sz_volume_ratio": np.nan, "cyb_volume_ratio": np.nan, "kcb50_volume_ratio": np.nan,
                    "a50_change": np.nan, "advance_decline_ratio": np.nan
                }
            except:
                # 如果获取样本失败，使用默认列名
                empty_factors = {col: np.nan for col in [
                    'sh_open_pct', 'sz_open_pct', 'cyb_open_pct', 'kcb50_open_pct',
                    'sh_gap_vs_high', 'sz_gap_vs_high', 'cyb_gap_vs_high', 'kcb50_gap_vs_high',
                    'sh_return_3d', 'sz_return_3d', 'cyb_return_3d', 'kcb50_return_3d',
                    'sh_volatility_5d', 'sz_volatility_5d', 'cyb_volatility_5d', 'kcb50_volatility_5d',
                    'sh_volume_ratio', 'sz_volume_ratio', 'cyb_volume_ratio', 'kcb50_volume_ratio',
                    'a50_change', 'advance_decline_ratio'
                ]}
                
            factors_cache[cache_key] = pd.DataFrame([empty_factors], index=[date_str])

    # 3. 合并所有因子数据
    all_factors = pd.concat(factors_cache.values())
    
    # 4. 合并到原始DataFrame
    result_df = combined_df.merge(
        all_factors,
        left_on="date_key",
        right_index=True,
        how="left"
    )
    
    return result_df

In [31]:
from date_utils import date
combined_df = add_market_factors(combined_df, date.get_previous_trade_date)
combined_df
# enhanced_df

处理大盘因子: 100%|██████████| 212/212 [02:17<00:00,  1.54it/s]


,id,date_key,stock_code,stock_name,block_category,block_codes,industry_code,max_block_category_rank,max_block_code_rank,max_industry_code_rank,...,sz_volatility_5d,sz_volume_ratio,cyb_return_3d,cyb_volatility_5d,cyb_volume_ratio,kcb50_return_3d,kcb50_volatility_5d,kcb50_volume_ratio,a50_change,advance_decline_ratio
0,1630,2024-09-02,000001.SZ,平安银行,"000003.BKDL,000038.BKDL,000023.BKDL,000037.BKDL","885694.DDBK,885867.DDBK,885745.DDBK,883300.DDB...",,7,2,-1,...,0.011632,1.591472,3.247767,0.011763,1.562755,2.938695,0.011674,1.798735,1.40,0.892278
1,16559,2024-09-02,000008.SZ,神州高铁,"000012.BKDL,000038.BKDL,000023.BKDL,000035.BKD...","886021.DDBK,885867.DDBK,885517.DDBK,883437.DDB...",,2,1,-1,...,0.011632,1.591472,3.247767,0.011763,1.562755,2.938695,0.011674,1.798735,1.40,0.892278
2,16304,2024-09-02,000026.SZ,飞亚达,"000023.BKDL,000014.BKDL,000010.BKDL,000017.BKDL","885420.DDBK,886021.DDBK,886072.DDBK,885595.DDB...",,10,21,-1,...,0.011632,1.591472,3.247767,0.011763,1.562755,2.938695,0.011674,1.798735,1.40,0.892278
3,16594,2024-09-02,000058.SZ,深 赛 格,"000012.BKDL,000038.BKDL,000023.BKDL,000035.BKD...","886021.DDBK,885574.DDBK,885729.DDBK,885697.DDB...",,2,1,-1,...,0.011632,1.591472,3.247767,0.011763,1.562755,2.938695,0.011674,1.798735,1.40,0.892278
4,9542,2024-09-02,000065.SZ,北方国际,"000008.BKDL,000038.BKDL,000026.BKDL,000035.BKD...","886045.DDBK,885996.DDBK,885840.DDBK,886021.DDB...",,6,2,-1,...,0.011632,1.591472,3.247767,0.011763,1.562755,2.938695,0.011674,1.798735,1.40,0.892278
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84761,34576,2025-07-21,605287.SH,德才股份,"000012.BKDL,000023.BKDL,000035.BKDL,000007.BKDL","885991.DDBK,886089.DDBK,885709.DDBK,885729.DDB...",,3,14,-1,...,0.008320,1.030455,1.883716,0.012911,1.013562,1.132150,0.005764,1.028301,0.68,0.503482
84762,38011,2025-07-21,605336.SH,帅丰电器,"000023.BKDL,000010.BKDL","885420.DDBK,886072.DDBK,885543.DDBK,885784.DDBK",,3,29,-1,...,0.008320,1.030455,1.883716,0.012911,1.013562,1.132150,0.005764,1.028301,0.68,0.503482
84763,34724,2025-07-21,605366.SH,宏柏新材,"000036.BKDL,000023.BKDL,000035.BKDL,000037.BKD...","885520.DDBK,885912.DDBK,886089.DDBK,885958.DDB...",,3,6,-1,...,0.008320,1.030455,1.883716,0.012911,1.013562,1.132150,0.005764,1.028301,0.68,0.503482
84764,37130,2025-07-21,605399.SH,晨光新材,"000036.BKDL,000035.BKDL,000037.BKDL,000022.BKD...","885520.DDBK,885912.DDBK,883303.DDBK,885929.DDB...",,1,2,-1,...,0.008320,1.030455,1.883716,0.012911,1.013562,1.132150,0.005764,1.028301,0.68,0.503482


In [32]:
# 创建龙虎榜数据缓存
lb_cache = {}

def get_lb_data(trade_date):
    """获取并缓存单日龙虎榜数据"""
    if trade_date not in lb_cache:
        try:
            df = pro.top_list(trade_date=trade_date)
            # 确保返回的DataFrame包含所需列
            if df is not None and not df.empty:
                lb_cache[trade_date] = df[['ts_code', 'reason']]
            else:
                lb_cache[trade_date] = pd.DataFrame(columns=['ts_code', 'reason'])
        except Exception as e:
            print(f"Error getting data for {trade_date}: {e}")
            lb_cache[trade_date] = pd.DataFrame(columns=['ts_code', 'reason'])
    return lb_cache[trade_date]

# 主处理函数
def process_data(combined_df):
    # 初始化结果列
    combined_df['last_day_on_list'] = 0
    combined_df['reason_last_day'] = 'null'
    combined_df['count_last_5_days'] = 0
    combined_df['last_reason_5_days'] = 'null'  # 新增列：最近5日最近一次上榜原因
    
    # 获取所有需要查询的日期
    all_dates = set()
    for date_key in combined_df['date_key'].unique():
        prev_dates = get_previous_trade_date_n(date_key, 5)
        all_dates.update(prev_dates)
    
    # 预加载龙虎榜数据
    for date in all_dates:
        # 确保日期格式正确（YYYYMMDD）
        formatted_date = date.replace('-', '') if '-' in date else date
        _ = get_lb_data(formatted_date)
    
    # 处理每一行数据
    for idx, row in combined_df.iterrows():
        date_key = row['date_key']
        stock_code = row['stock_code']
        
        # 1. 获取前1个交易日
        prev_1_day = get_previous_trade_date_n(date_key, 1)
        if not prev_1_day:
            continue
            
        prev_date = prev_1_day[0]
        formatted_prev_date = prev_date.replace('-', '') if '-' in prev_date else prev_date
        lb_data = get_lb_data(formatted_prev_date)
        
        # 2. 检查前1日是否上榜
        if not lb_data.empty:
            stock_data = lb_data[lb_data['ts_code'] == stock_code]
            if not stock_data.empty:
                combined_df.at[idx, 'last_day_on_list'] = 1
                combined_df.at[idx, 'reason_last_day'] = stock_data.iloc[0]['reason']
        
        # 3. 统计近5个交易日上榜次数和最近原因
        prev_5_days = get_previous_trade_date_n(date_key, 5)
        count = 0
        last_reason = 'null'  # 用于记录最近一次上榜原因
        
        # 按时间顺序遍历（最近日期在前）
        for trade_date in prev_5_days:
            formatted_date = trade_date.replace('-', '') if '-' in trade_date else trade_date
            lb_data = get_lb_data(formatted_date)
            
            if not lb_data.empty:
                stock_data = lb_data[lb_data['ts_code'] == stock_code]
                if not stock_data.empty:
                    count += 1
                    # 记录最近一次上榜原因（因为遍历顺序是最近日期在前）
                    if last_reason == 'null':
                        last_reason = stock_data.iloc[0]['reason']
        
        combined_df.at[idx, 'count_last_5_days'] = count
        combined_df.at[idx, 'last_reason_5_days'] = last_reason
    
    return combined_df

# 使用示例
# 假设combined_df是您的输入DataFrame
# 包含列：['date_key', 'stock_code']
combined_df = process_data(combined_df.copy())

In [33]:
combined_df

,id,date_key,stock_code,stock_name,block_category,block_codes,industry_code,max_block_category_rank,max_block_code_rank,max_industry_code_rank,...,cyb_volume_ratio,kcb50_return_3d,kcb50_volatility_5d,kcb50_volume_ratio,a50_change,advance_decline_ratio,last_day_on_list,reason_last_day,count_last_5_days,last_reason_5_days
0,1630,2024-09-02,000001.SZ,平安银行,"000003.BKDL,000038.BKDL,000023.BKDL,000037.BKDL","885694.DDBK,885867.DDBK,885745.DDBK,883300.DDB...",,7,2,-1,...,1.562755,2.938695,0.011674,1.798735,1.40,0.892278,0,null,0,null
1,16559,2024-09-02,000008.SZ,神州高铁,"000012.BKDL,000038.BKDL,000023.BKDL,000035.BKD...","886021.DDBK,885867.DDBK,885517.DDBK,883437.DDB...",,2,1,-1,...,1.562755,2.938695,0.011674,1.798735,1.40,0.892278,0,null,0,null
2,16304,2024-09-02,000026.SZ,飞亚达,"000023.BKDL,000014.BKDL,000010.BKDL,000017.BKDL","885420.DDBK,886021.DDBK,886072.DDBK,885595.DDB...",,10,21,-1,...,1.562755,2.938695,0.011674,1.798735,1.40,0.892278,0,null,0,null
3,16594,2024-09-02,000058.SZ,深 赛 格,"000012.BKDL,000038.BKDL,000023.BKDL,000035.BKD...","886021.DDBK,885574.DDBK,885729.DDBK,885697.DDB...",,2,1,-1,...,1.562755,2.938695,0.011674,1.798735,1.40,0.892278,0,null,0,null
4,9542,2024-09-02,000065.SZ,北方国际,"000008.BKDL,000038.BKDL,000026.BKDL,000035.BKD...","886045.DDBK,885996.DDBK,885840.DDBK,886021.DDB...",,6,2,-1,...,1.562755,2.938695,0.011674,1.798735,1.40,0.892278,0,null,0,null
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84761,34576,2025-07-21,605287.SH,德才股份,"000012.BKDL,000023.BKDL,000035.BKDL,000007.BKDL","885991.DDBK,886089.DDBK,885709.DDBK,885729.DDB...",,3,14,-1,...,1.013562,1.132150,0.005764,1.028301,0.68,0.503482,0,null,0,null
84762,38011,2025-07-21,605336.SH,帅丰电器,"000023.BKDL,000010.BKDL","885420.DDBK,886072.DDBK,885543.DDBK,885784.DDBK",,3,29,-1,...,1.013562,1.132150,0.005764,1.028301,0.68,0.503482,0,null,3,非ST、*ST和S证券连续三个交易日内收盘价格跌幅偏离值累计达到20%的证券
84763,34724,2025-07-21,605366.SH,宏柏新材,"000036.BKDL,000023.BKDL,000035.BKDL,000037.BKD...","885520.DDBK,885912.DDBK,886089.DDBK,885958.DDB...",,3,6,-1,...,1.013562,1.132150,0.005764,1.028301,0.68,0.503482,0,null,1,有价格涨跌幅限制的日收盘价格跌幅偏离值达到7%的前五只证券
84764,37130,2025-07-21,605399.SH,晨光新材,"000036.BKDL,000035.BKDL,000037.BKDL,000022.BKD...","885520.DDBK,885912.DDBK,883303.DDBK,885929.DDB...",,1,2,-1,...,1.013562,1.132150,0.005764,1.028301,0.68,0.503482,0,null,2,有价格涨跌幅限制的日收盘价格跌幅偏离值达到7%的前五只证券


In [36]:

# 设置 Pandas 显示选项，不限制列的显示数量
pd.set_option('display.max_columns', None)

# 打印 DataFrame 的全部列名
combined_df.columns

Index(['id', 'date_key', 'stock_code', 'stock_name', 'block_category',
       'block_codes', 'industry_code', 'max_block_category_rank',
       'max_block_code_rank', 'max_industry_code_rank',
       ...
       'cyb_volume_ratio', 'kcb50_return_3d', 'kcb50_volatility_5d',
       'kcb50_volume_ratio', 'a50_change', 'advance_decline_ratio',
       'last_day_on_list', 'reason_last_day', 'count_last_5_days',
       'last_reason_5_days'],
      dtype='object', length=289)

In [48]:
nan_counts = combined_df.isna().sum()
# 筛选出至少含一个 NaN 的列
nan_columns = nan_counts[nan_counts > 0].index.tolist()

In [ ]:
# 写入 CSV 文件
combined_df.to_csv('combined_data.csv',  # 文件名
                  index=False,          # 不保存行索引
                  na_rep='NaN')         # 将 NaN 替换为字符串 'NaN' 保存

: 

In [1]:
import pandas as pd

# 从 CSV 加载 DataFrame
loaded_df = pd.read_csv('combined_data.csv', 
                        na_values=['NaN', 'nan', 'NAN', 'null', 'NULL', ''],  # 识别为 NaN 的字符串
                        keep_default_na=True)  # 同时识别系统默认的 NaN 表示

# 验证加载结果
print(f"成功加载数据，形状: {loaded_df.shape}")
print(f"列名: {loaded_df.columns.tolist()}")

成功加载数据，形状: (84766, 289)
列名: ['id', 'date_key', 'stock_code', 'stock_name', 'block_category', 'block_codes', 'industry_code', 'max_block_category_rank', 'max_block_code_rank', 'max_industry_code_rank', 'is_bottom', 'is_broken_plate', 'is_down_broken', 'is_fall', 'is_first_down_broken', 'is_first_up_broken', 'is_gestation_line', 'is_half', 'is_high', 'is_highest', 'is_long_shadow', 'is_low', 'is_medium', 'is_meso', 'is_plummet', 'is_pre_st', 'is_small_high_open', 'is_up_broken', 'is_weak', 'first_limit_up_days', 'jsjl', 'cjs', 'xcjw', 'jssb', 'open_pct_rate', 'open_price', 'close_price', 'pre_close_price', 'next_day_open_price', 'next_day_close_price', 'next_day_high_price_open_10mins', 'next_day_low_price_open_10mins', 'next_day_high_price', 'next_day_low_price', 'in_premarket', 'in_premarket_match', 'mod_code', 'mod_name', 'mod_short_line_score', 'mod_short_line_score_change', 'mod_short_line_rank', 'mod_trend_score', 'mod_trend_score_change', 'mod_trend_rank', 'env_json_info', 'xcjw_v

C:\Users\Administrator\AppData\Local\Temp\ipykernel_79424\3365854924.py:4: DtypeWarning: Columns (70,71,183,193,201) have mixed types. Specify dtype option on import or set low_memory=False.
  loaded_df = pd.read_csv('combined_data.csv',


包含 NaN 的列及其缺失值比例：
                                                 NaN比例(%)
industry_code                                      100.00
mod_code                                           100.00
mod_name                                           100.00
reason_last_day                                     94.54
htyk                                                94.52
cgyk                                                86.34
last_reason_5_days                                  84.02
top1_predays_avgrank_ppp_T_track_T                  18.35
top2_predays_avgrank_ppp_T_track_T                  18.35
top3_predays_avgrank_ppp_T_track_T                  18.35
top1_predays_avgrank_ppp_F_track_T                  14.47
top3_predays_avgrank_ppp_F_track_T                  14.47
top2_predays_avgrank_ppp_F_track_T                  14.47
top3_avg_category_predays_avgrank_ppp_F_track_T      9.26
top1_category_predays_avgrank_ppp_F_track_T          9.26
top1_category_predays_avgrank_ppp_T_track_T          9

In [44]:
import sys
sys.path.append(r"D:\workspace\TradeX\ezMoney")
from common import constants
import concurrent.futures
from functools import partial

# 获取交易日历（用于计算前后交易日）
def get_trade_cal(start_date, end_date):
    """获取交易日历并缓存"""
    cal = pro.trade_cal(exchange='', start_date=start_date, end_date=end_date)
    cal = cal[cal['is_open'] == 1]
    return cal['cal_date'].tolist()

# 创建价格数据缓存
price_cache = {}

def get_daily_data(ts_code, trade_date):
    """使用xtdata获取单只股票单日行情数据（带缓存）"""
    key = (ts_code, trade_date)
    if key not in price_cache:
        try:
            # 使用xtdata获取数据
            # 注意：xtdata的日期格式为YYYYMMDD，股票代码格式为000001.SH/SZ
            # 确定市场后缀
            xt_symbol = ts_code
            xtdata.download_history_data(xt_symbol,'1d', trade_date, trade_date)
            # 获取单日数据
            data = xtdata.get_market_data(
                field_list=['open', 'close', 'pre_close'],
                stock_list=[xt_symbol],
                period='1d',
                start_time=trade_date,
                end_time=trade_date,
                fill_data=True
            )

            print(data)
            # 修正：检查返回的数据类型
            if isinstance(data, dict) and xt_symbol in data:
                # 从字典中提取数据
                stock_data = data[xt_symbol]
                if stock_data and 'open' in stock_data and 'close' in stock_data and 'pre_close' in stock_data:
                    price_cache[key] = {
                        'open': stock_data['open'],
                        'close': stock_data['close'],
                        'pre_close': stock_data['pre_close']
                    }
                else:
                    print(f"Warning: Incomplete data for {ts_code} on {trade_date}")
                    price_cache[key] = None
            else:
                print(f"Warning: No data for {ts_code} on {trade_date}")
                price_cache[key] = None
            
        except Exception as e:
            print(f"Error getting xtdata for {ts_code} on {trade_date}: {e}")
            price_cache[key] = None
    return price_cache[key]

def pre_download_data(combined_df):
    """预下载所有需要的数据"""
    # 获取日期范围
    min_date = combined_df['date_key'].min().replace('-', '')
    max_date = combined_df['date_key'].max().replace('-', '')
    
    # 扩展日期范围（前后各加30天以确保覆盖）
    min_date_ext = (pd.to_datetime(min_date) - pd.Timedelta(days=30)).strftime('%Y%m%d')
    max_date_ext = (pd.to_datetime(max_date) + pd.Timedelta(days=30)).strftime('%Y%m%d')
    
    # 获取交易日历
    trade_dates = get_trade_cal(min_date_ext, max_date_ext)
    trade_dates.sort()  # 日期升序排列
    
    # 创建日期索引映射
    date_to_idx = {date: idx for idx, date in enumerate(trade_dates)}
    
    # 获取下一个交易日的函数
    def get_next_trade_date(date, n=1):
        """获取指定日期后第n个交易日"""
        date = date.replace('-', '')
        if date in date_to_idx:
            idx = date_to_idx[date] + n
            return trade_dates[idx] if idx < len(trade_dates) else None
        return None
    
    # 收集所有需要下载的数据点
    download_tasks = []
    
    for idx, row in combined_df.iterrows():
        date0 = row['date_key'].replace('-', '')  # 当天
        stock_code = row['stock_code']
        
        # 添加当天数据
        download_tasks.append((stock_code, date0))
        
        # 添加前一天数据
        prev_date = get_next_trade_date(date0, -1)
        if prev_date:
            download_tasks.append((stock_code, prev_date))
        
        # 添加后续交易日数据
        for n in [1, 2, 3]:
            next_date = get_next_trade_date(date0, n)
            if next_date:
                download_tasks.append((stock_code, next_date))
    
    # 去重
    download_tasks = list(set(download_tasks))
    print(f"准备下载 {len(download_tasks)} 个数据点...")
    
    # 使用线程池下载数据
    with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
        # 创建进度条
        progress = tqdm(total=len(download_tasks), desc="下载数据")
        
        # 定义下载函数
        def download_task(task):
            stock_code, trade_date = task
            result = get_daily_data(stock_code, trade_date)
            progress.update(1)
            return result
        
        # 提交任务
        futures = [executor.submit(download_task, task) for task in download_tasks]
        
        # 等待所有任务完成
        for future in concurrent.futures.as_completed(futures):
            pass  # 不需要处理结果，数据已缓存
    
    progress.close()
    print("数据下载完成")

# 处理单行数据的函数
def process_row(row, date_to_idx, trade_dates):
    """处理单行数据"""
    # 创建结果字典
    result = {
        'label1': np.nan,
        'label2': np.nan,
        'label3': np.nan,
        'preclose': np.nan,
        'open0': np.nan,
        'close0': np.nan,
        'open1': np.nan,
        'close1': np.nan,
        'open2': np.nan,
        'close2': np.nan,
        'open3': np.nan,
        'close3': np.nan
    }
    
    date0 = row['date_key'].replace('-', '')  # 当天
    stock_code = row['stock_code']
    
    # 获取下一个交易日的函数
    def get_next_trade_date(date, n=1):
        """获取指定日期后第n个交易日"""
        date = date.replace('-', '')
        if date in date_to_idx:
            idx = date_to_idx[date] + n
            return trade_dates[idx] if idx < len(trade_dates) else None
        return None
    
    # 1. 获取当天数据
    data0 = get_daily_data(stock_code, date0)
    if data0 is None:
        return result
    
    # 保存当天价格数据
    result['open0'] = data0['open']
    result['close0'] = data0['close']
        
    # 2. 获取前一天数据
    prev_date = get_next_trade_date(date0, -1)
    if not prev_date:
        return result
        
    data_prev = get_daily_data(stock_code, prev_date)
    if data_prev is None:
        return result
    
    # 保存前一日收盘价
    result['preclose'] = data_prev['close']
        
    # 3. 获取后续交易日数据
    date1 = get_next_trade_date(date0, 1)  # 第二天
    date2 = get_next_trade_date(date0, 2)  # 第三天
    date3 = get_next_trade_date(date0, 3)  # 第四天
    
    # 获取后续交易日价格数据并保存
    if date1 is not None:
        data1 = get_daily_data(stock_code, date1)
        if data1 is not None:
            result['open1'] = data1['open']
            result['close1'] = data1['close']
    
    if date2 is not None:
        data2 = get_daily_data(stock_code, date2)
        if data2 is not None:
            result['open2'] = data2['open']
            result['close2'] = data2['close']
    
    if date3 is not None:
        data3 = get_daily_data(stock_code, date3)
        if data3 is not None:
            result['open3'] = data3['open']
            result['close3'] = data3['close']
    
    # 4. 计算涨跌停价
    limit_down, limit_up = constants.get_limit_price(data_prev['close'], stock_code)
    
    # 5. 计算label1
    # 涨停或跌停时使用第二天开盘价计算
    use_second_day = False
    if data0['close'] >= limit_up - 0.001 or data0['close'] <= limit_down + 0.001:
        use_second_day = True
    
    if use_second_day:
        if date1 is not None and 'open1' in result and not pd.isna(result['open1']):
            open1 = result['open1']
            # 使用第二天开盘价计算
            ret1 = (open1 - data_prev['close']) / data_prev['close']  # 相对前收盘
            ret2 = (open1 - data0['open']) / data0['open']  # 相对当天开盘
            result['label1'] = (ret1 + ret2) / 2
    else:
        # 正常情况
        ret1 = (data0['close'] - data_prev['close']) / data_prev['close']  # 相对前收盘
        ret2 = (data0['close'] - data0['open']) / data0['open']  # 相对当天开盘
        result['label1'] = (ret1 + ret2) / 2
    
    # 6. 计算label2
    if date1 is not None and 'close1' in result and not pd.isna(result['close1']):
        close1 = result['close1']
        open0 = data0['open']
        
        # 计算第二天的涨跌停价
        limit_down1, limit_up1 = constants.get_limit_price(data0['close'], stock_code)
        
        # 第二天涨停或跌停时使用第三天开盘价计算
        use_third_day = False
        if close1 >= limit_up1 - 0.001 or close1 <= limit_down1 + 0.001:
            use_third_day = True
        
        if use_third_day:
            if date2 is not None and 'open2' in result and not pd.isna(result['open2']):
                open2 = result['open2']
                result['label2'] = (open2 - open0) / open0
        else:
            # 正常情况
            result['label2'] = (close1 - open0) / open0
    
    # 7. 计算label3
    if date2 is not None and 'close2' in result and not pd.isna(result['close2']):
        close2 = result['close2']
        open0 = data0['open']
        
        # 计算第三天的涨跌停价（需要第二天收盘价）
        if date1 is not None and 'close1' in result and not pd.isna(result['close1']):
            close1 = result['close1']
            limit_down2, limit_up2 = constants.get_limit_price(close1, stock_code)
            
            # 第三天涨停或跌停时使用第四天开盘价计算
            use_fourth_day = False
            if close2 >= limit_up2 - 0.001 or close2 <= limit_down2 + 0.001:
                use_fourth_day = True
            
            if use_fourth_day:
                if date3 is not None and 'open3' in result and not pd.isna(result['open3']):
                    open3 = result['open3']
                    result['label3'] = (open3 - open0) / open0
            else:
                # 正常情况
                result['label3'] = (close2 - open0) / open0
    
    return result

# 主处理函数
def calculate_labels(combined_df):
    # 获取日期范围
    min_date = combined_df['date_key'].min().replace('-', '')
    max_date = combined_df['date_key'].max().replace('-', '')
    
    # 扩展日期范围（前后各加30天以确保覆盖）
    min_date_ext = (pd.to_datetime(min_date) - pd.Timedelta(days=30)).strftime('%Y%m%d')
    max_date_ext = (pd.to_datetime(max_date) + pd.Timedelta(days=30)).strftime('%Y%m%d')
    
    # 获取交易日历
    trade_dates = get_trade_cal(min_date_ext, max_date_ext)
    trade_dates.sort()  # 日期升序排列
    
    # 创建日期索引映射
    date_to_idx = {date: idx for idx, date in enumerate(trade_dates)}
    
    # 1. 预下载所有需要的数据
    pre_download_data(combined_df)
    
    # 2. 使用多线程处理每一行数据
    results = []
    
    # 创建处理函数的部分应用
    process_func = partial(process_row, date_to_idx=date_to_idx, trade_dates=trade_dates)
    
    # 使用线程池处理
    with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
        # 提交任务
        futures = {executor.submit(process_func, row): idx for idx, row in combined_df.iterrows()}
        
        # 创建进度条
        progress = tqdm(total=len(combined_df), desc="处理数据")
        
        # 处理完成的任务
        for future in concurrent.futures.as_completed(futures):
            idx = futures[future]
            try:
                result = future.result()
                results.append((idx, result))
            except Exception as e:
                print(f"Error processing row {idx}: {e}")
                results.append((idx, {
                    'label1': np.nan, 'label2': np.nan, 'label3': np.nan,
                    'preclose': np.nan, 'open0': np.nan, 'close0': np.nan,
                    'open1': np.nan, 'close1': np.nan, 'open2': np.nan, 'close2': np.nan,
                    'open3': np.nan, 'close3': np.nan
                }))
            progress.update(1)
        
        progress.close()
    
    # 3. 将结果合并回DataFrame
    # 先按原始索引排序
    results.sort(key=lambda x: x[0])
    
    # 提取结果数据
    result_data = [r[1] for r in results]
    
    # 将结果添加到DataFrame
    combined_df['label1'] = [r['label1'] for r in result_data]
    combined_df['label2'] = [r['label2'] for r in result_data]
    combined_df['label3'] = [r['label3'] for r in result_data]
    combined_df['preclose'] = [r['preclose'] for r in result_data]
    combined_df['open0'] = [r['open0'] for r in result_data]
    combined_df['close0'] = [r['close0'] for r in result_data]
    combined_df['open1'] = [r['open1'] for r in result_data]
    combined_df['close1'] = [r['close1'] for r in result_data]
    combined_df['open2'] = [r['open2'] for r in result_data]
    combined_df['close2'] = [r['close2'] for r in result_data]
    combined_df['open3'] = [r['open3'] for r in result_data]
    combined_df['close3'] = [r['close3'] for r in result_data]
    
    return combined_df


In [45]:
combined_df.tail(20)

,id,date_key,stock_code,stock_name,block_category,block_codes,industry_code,max_block_category_rank,max_block_code_rank,max_industry_code_rank,is_bottom,is_broken_plate,is_down_broken,is_fall,is_first_down_broken,is_first_up_broken,is_gestation_line,is_half,is_high,is_highest,is_long_shadow,is_low,is_medium,is_meso,is_plummet,is_pre_st,is_small_high_open,is_up_broken,is_weak,first_limit_up_days,jsjl,cjs,xcjw,jssb,open_pct_rate,open_price,close_price,pre_close_price,next_day_open_price,next_day_close_price,next_day_high_price_open_10mins,next_day_low_price_open_10mins,next_day_high_price,next_day_low_price,in_premarket,in_premarket_match,mod_code,mod_name,mod_short_line_score,mod_short_line_score_change,mod_short_line_rank,mod_trend_score,mod_trend_score_change,mod_trend_rank,env_json_info,xcjw_v2,jssb_v2,cjs_v2,is_strength_high,is_strength_middle,is_strength_low,is_strength_increase,is_strength_reduct,short_line_score,short_line_score_change,jsjl_block,jssb_block,cjs_block,direction_cjs_v2,circulation_market_value,cgyk,htyk,cgyk_value,htyk_value,is_middle_high_open,is_large_high_open,is_small_low_open,is_middle_low_open,is_large_low_open,stock_rank_dx,stock_rank_jl,stock_rank_jw,stock_rank_qb,top1_rank_ppp_T_track_T,top2_rank_ppp_T_track_T,top3_rank_ppp_T_track_T,top3_avgrank_ppp_T_track_T,top1_category_rank_ppp_T_track_T,top3_avg_category_rank_ppp_T_track_T,top1_predays_avgrank_ppp_T_track_T,top2_predays_avgrank_ppp_T_track_T,top3_predays_avgrank_ppp_T_track_T,top1_category_predays_avgrank_ppp_T_track_T,top3_avg_category_predays_avgrank_ppp_T_track_T,top1_rank_ppp_T_track_F,top2_rank_ppp_T_track_F,top3_rank_ppp_T_track_F,top3_avgrank_ppp_T_track_F,top1_category_rank_ppp_T_track_F,top3_avg_category_rank_ppp_T_track_F,top1_predays_avgrank_ppp_T_track_F,top2_predays_avgrank_ppp_T_track_F,top3_predays_avgrank_ppp_T_track_F,top1_category_predays_avgrank_ppp_T_track_F,top3_avg_category_predays_avgrank_ppp_T_track_F,top1_rank_ppp_F_track_T,top2_rank_ppp_F_track_T,top3_rank_ppp_F_track_T,top3_avgrank_ppp_F_track_T,top1_category_rank_ppp_F_track_T,top3_avg_category_rank_ppp_F_track_T,top1_predays_avgrank_ppp_F_track_T,top2_predays_avgrank_ppp_F_track_T,top3_predays_avgrank_ppp_F_track_T,top1_category_predays_avgrank_ppp_F_track_T,top3_avg_category_predays_avgrank_ppp_F_track_T,top1_rank_ppp_F_track_F,top2_rank_ppp_F_track_F,top3_rank_ppp_F_track_F,top3_avgrank_ppp_F_track_F,top1_category_rank_ppp_F_track_F,top3_avg_category_rank_ppp_F_track_F,top1_predays_avgrank_ppp_F_track_F,top2_predays_avgrank_ppp_F_track_F,top3_predays_avgrank_ppp_F_track_F,top1_category_predays_avgrank_ppp_F_track_F,top3_avg_category_predays_avgrank_ppp_F_track_F,prev_price_change_5d,prev_price_change_7d,prev_price_change_10d,prev_15d_high,prev_break_15d_high,prev_limit_up,prev_limit_down,prev_one_word_limit,prev_limit_up_count_5d,prev_limit_up_count_7d,prev_limit_up_count_10d,prev_limit_down_count_5d,prev_limit_down_count_7d,prev_limit_down_count_10d,prev_one_word_count_5d,prev_one_word_count_7d,prev_one_word_count_10d,prev_vol_ma5,prev_vol_ma7,prev_vol_ma10,prev_volume_ratio_5d,prev_volume_ratio_7d,prev_volume_ratio_10d,prev_limit_streak,prev_consecutive_limit_5d,prev_consecutive_limit_7d,prev_consecutive_limit_10d,prev_amplitude,prev_ma_trend_3d,prev_ma_trend_5d,prev_ma_trend_7d,prev_is_yang,prev_is_yin,prev_yang_count_5d,prev_yang_count_7d,prev_yang_count_10d,prev_yin_count_5d,prev_yin_count_7d,prev_yin_count_10d,prev_adx_5,prev_adx_7,prev_adx_10,prev_boll_width_5,prev_boll_width_7,prev_boll_width_10,prev_high_5d,prev_high_7d,prev_high_10d,prev_pullback_5d,prev_pullback_7d,prev_pullback_10d,prev_one_word_failed,prev_limit_up_open,prev_failed_limit,prev_volume_failed_limit,prev_limit_up_prev,prev_broken_limit,prev_volume_broken_limit,prev_failed_limit_yin,prev_failed_limit_break_ma5,prev_volume_down,prev_ma5_slope,prev_ma3,prev_ma3_slope,prev_trend_reversal,prev_uptrend,prev_trend_pullback,prev_pullback_ratio,prev_range_15d,prev_consolidation,prev_volume_limi

In [46]:

# 使用示例
# 假设combined_df是您的输入DataFrame
# 包含列：['date_key', 'stock_code']
test_df = calculate_labels(combined_df.tail(20).copy())

# 查看结果
print(test_df[['date_key', 'stock_code', 'preclose', 'open0', 'close0', 
                'open1', 'close1', 'open2', 'close2', 'open3', 'close3',
                'label1', 'label2', 'label3']].head())

准备下载 100 个数据点...


下载数据:   2%|▏         | 2/100 [00:00<00:05, 17.09it/s]

{'open':            20250718
605259.SH      24.1, 'close':            20250718
605259.SH     24.12}{'open':            20250723
603959.SH       5.4, 'close':            20250723
603959.SH      5.27}

{'open':            20250724
605018.SH      12.7, 'close':            20250724
605018.SH     12.73}
{'open':            20250718
605488.SH      32.4, 'close':            20250718
605488.SH     32.12}
{'open':            20250721
603960.SH     19.35, 'close':            20250721
603960.SH     19.49}
{'open':            20250718
603999.SH      6.65, 'close':            20250718
603999.SH      6.68}


下载数据:  17%|█▋        | 17/100 [00:00<00:02, 28.26it/s]

{'open':            20250722
603969.SH       8.6, 'close':            20250722
603969.SH      9.13}{'open':            20250718
605399.SH     14.24, 'close':            20250718
605399.SH     13.99}

{'open':            20250718
603948.SH     21.66, 'close':            20250718
603948.SH      21.8}
{'open':            20250718
605178.SH      23.0, 'close':            20250718
605178.SH     23.26}
{'open':            20250723
605259.SH     24.17, 'close':            20250723
605259.SH      23.9}
{'open':            20250724
605287.SH      15.5, 'close':            20250724
605287.SH     15.49}
{'open':            20250724
605068.SH     17.93, 'close':            20250724
605068.SH     18.11}
{'open':            20250721
605178.SH     23.62, 'close':            20250721
605178.SH     24.19}
{'open':            20250723
605488.SH      31.3, 'close':            20250723
605488.SH     31.39}
{'open':            20250718
605336.SH     17.09, 'close':            20250718
605336.SH     17.16}


...

{'open':            20250723
605277.SH      26.2, 'close':            20250723
605277.SH     24.93}
{'open':            20250724
605069.SH     10.84, 'close':            20250724
605069.SH     10.86}
{'open':            20250721
603969.SH      7.88, 'close':            20250721
603969.SH       8.3}
{'open':            20250724
603990.SH     16.18, 'close':            20250724
603990.SH     16.25}
{'open':            20250722
603960.SH     19.53, 'close':            20250722
603960.SH     19.22}
{'open':            20250718
603959.SH      5.51, 'close':            20250718
603959.SH      5.58}
{'open':            20250723
603969.SH      9.57, 'close':            20250723
603969.SH      8.35}
{'open':            20250724
603992.SH     24.49, 'close':            20250724
603992.SH     26.34}


下载数据: 100%|██████████| 100/100 [00:02<00:00, 35.66it/s]


{'open':            20250722
605277.SH      23.9, 'close':            20250722
605277.SH     26.19}
{'open':            20250724
603977.SH     13.08, 'close':            20250724
603977.SH     13.18}
数据下载完成


处理数据: 100%|██████████| 20/20 [00:00<00:00, 19996.68it/s]

         date_key stock_code  preclose  open0  close0  open1  close1  open2  \
84746  2025-07-21  603948.SH       NaN    NaN     NaN    NaN     NaN    NaN   
84747  2025-07-21  603959.SH       NaN    NaN     NaN    NaN     NaN    NaN   
84748  2025-07-21  603960.SH       NaN    NaN     NaN    NaN     NaN    NaN   
84749  2025-07-21  603967.SH       NaN    NaN     NaN    NaN     NaN    NaN   
84750  2025-07-21  603969.SH       NaN    NaN     NaN    NaN     NaN    NaN   

       close2  open3  close3  label1  label2  label3  
84746     NaN    NaN     NaN     NaN     NaN     NaN  
84747     NaN    NaN     NaN     NaN     NaN     NaN  
84748     NaN    NaN     NaN     NaN     NaN     NaN  
84749     NaN    NaN     NaN     NaN     NaN     NaN  
84750     NaN    NaN     NaN     NaN     NaN     NaN  


In [119]:
result_df[['date_key', 'stock_code', 'preclose', 'open0', 'close0', 
                'open1', 'close1', 'open2', 'close2', 'open3', 'close3',
                'label1', 'label2', 'label3']]

,date_key,stock_code,preclose,open0,close0,open1,close1,open2,close2,open3,close3,label1,label2,label3
84746,2025-07-21,603948.SH,21.80,22.10,22.24,22.40,22.00,21.91,21.82,21.77,21.97,0.013259,-0.004525,-0.012670
84747,2025-07-21,603959.SH,5.58,5.40,5.47,5.47,5.45,5.40,5.27,5.25,5.28,-0.003375,0.009259,-0.024074
84748,2025-07-21,603960.SH,19.35,19.35,19.49,19.53,19.22,19.20,19.03,19.02,19.26,0.007235,-0.006718,-0.016537
84749,2025-07-21,603967.SH,11.32,11.32,11.40,11.42,11.39,11.43,11.58,11.71,11.84,0.007067,0.006184,0.022968
84750,2025-07-21,603969.SH,7.73,7.88,8.30,8.60,9.13,9.57,8.35,8.35,8.57,0.063519,0.214467,0.059645
84751,2025-07-21,603977.SH,12.30,12.92,13.53,13.42,13.93,14.02,13.00,13.08,13.18,0.064878,0.078173,0.006192
84752,2025-07-21,603990.SH,16.92,17.33,16.98,16.81,16.50,16.50,16.17,16.18,16.25,-0.008325,-0.047894,-0.066936
84753,2025-07-21,603992.SH,21.42,21.80,22.32,22.32,24.05,23.66,24.15,24.49,26.34,0.032935,0.103211,0.107798
84754,2025-07-21,603999.SH,6.68,6.65,6.75,6.75,6.69,6.70,6.57,6.56,6.70,0.012758,0.006015,-0.012030
84755,2025-07-21,605018.SH,13.15,13.00,13.20,13.10,12.92,12.85,12.75,12.70,12.73,0.009593,-0.006154,-0.019231
